In [1]:
#import basic packages
import pandas as pd
import numpy as np
import os
from pathlib import Path

from datetime import datetime, timedelta
import time

import gc
import copy

import json
import pickle

import pyarrow.parquet as pq
import pyarrow as pa

 
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score
from sklearn.model_selection import StratifiedKFold, KFold

from sklearn.preprocessing import label_binarize


In [2]:
#import pytorch 
import torch
from torch import nn
from tqdm import tqdm
import torch.optim as optim
from torch.nn import CrossEntropyLoss, MSELoss, BCELoss

In [3]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

import warnings
warnings.filterwarnings("ignore")

import pytorch_lightning as pl
random_seed=1234
pl.seed_everything(random_seed)

Global seed set to 1234


1234

## configs

In [4]:
# train_file = r'/kaggle/input/amex-agg-data-rev2/agg_train_all_rev2_rev.parquet'
train_file = r'amex/agg_v3/agg_train_all_small.parquet'

submission_file = '/kaggle/input/amex-default-prediction/sample_submission.csv'
test_file = '/kaggle/input/amex-agg-data-v3/agg_test_all_small.parquet'

In [5]:

id_feats = ['customer_ID']
#date_col =  'S_2'
target_col = 'target'

## amex metric

In [13]:
# @yunchonggan's fast metric implementation
# From https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
# https://www.kaggle.com/code/ambrosm/amex-lightgbm-quickstart
def amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)


In [ ]:
# @yunchonggan's fast metric implementation
# From https://www.kaggle.com/competitions/amex-default-prediction/discussion/328020
# https://www.kaggle.com/code/ambrosm/amex-lightgbm-quickstart
def amex_metric(y_true: np.array, y_pred: np.array) -> float:

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)


## LightGBM

In [16]:
import lightgbm as lgb
def train_trees(X_train, y_train, num_round=100, params={} ):
    
    params['verbosity'] = -1
    dtrain = lgb.Dataset(X_train, y_train)
    
    tree_model = lgb.train(params,
                dtrain,
                num_boost_round=num_round)
    
    
    del dtrain
    gc.collect()

    return tree_model

In [21]:
%%time

df = pd.read_parquet(train_file, columns=['customer_ID', 'target'] + feats, engine='pyarrow')
#df=pd.read_parquet(train_file, engine='pyarrow')

binary_params = [
{'boosting_type': 'gbdt', 'colsample_bytree': 0.1, 'learning_rate': 0.03, 'max_bin': 8950, 'max_depth': 8, 'metric': 'auc', 'min_child_samples': 2550, 'min_data_in_bin': 4450, 'n_estimators': 1950, 'n_jobs': 4, 'num_leaves': 56, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 10, 'reg_lambda': 0.001, 'subsample': 0.75, 'subsample_freq': 13}, 
{'boosting_type': 'gbdt', 'colsample_bytree': 0.1, 'learning_rate': 0.02, 'max_bin': 8500, 'max_depth': 22, 'metric': 'auc', 'min_child_samples': 600, 'min_data_in_bin': 3400, 'n_estimators': 1100, 'n_jobs': 4, 'num_leaves': 236, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 0.0001, 'reg_lambda': 10, 'subsample': 0.7, 'subsample_freq': 73}, 
{'boosting_type': 'gbdt', 'colsample_bytree': 0.1, 'learning_rate': 0.03, 'max_bin': 3600, 'max_depth': 17, 'metric': 'auc', 'min_child_samples': 300, 'min_data_in_bin': 2850, 'n_estimators': 1100, 'n_jobs': 4, 'num_leaves': 96, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 0.5, 'reg_lambda': 10, 'subsample': 0.8, 'subsample_freq': 19}, 
{'boosting_type': 'gbdt', 'colsample_bytree': 0.8, 'learning_rate': 0.03, 'max_bin': 9500, 'max_depth': 17, 'metric': 'auc', 'min_child_samples': 3150, 'min_data_in_bin': 600, 'n_estimators': 1500, 'n_jobs': 4, 'num_leaves': 126, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 1, 'reg_lambda': 10, 'subsample': 0.95, 'subsample_freq': 29},
{'boosting_type': 'gbdt', 'colsample_bytree': 0.75, 'learning_rate': 0.03, 'max_bin': 1900, 'max_depth': 17, 'metric': 'auc', 'min_child_samples': 3000, 'min_data_in_bin': 600, 'n_estimators': 1650, 'n_jobs': 4, 'num_leaves': 291, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 10, 'reg_lambda': 10, 'subsample': 0.7, 'subsample_freq': 37},
{'boosting_type': 'gbdt', 'colsample_bytree': 0.75, 'learning_rate': 0.02, 'max_bin': 1900, 'max_depth': 30, 'metric': 'auc', 'min_child_samples': 3000, 'min_data_in_bin': 2850, 'n_estimators': 1250, 'n_jobs': 4, 'num_leaves': 291, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 0.0001, 'reg_lambda': 0.001, 'subsample': 0.8, 'subsample_freq': 97},
{'boosting_type': 'gbdt', 'colsample_bytree': 0.1, 'learning_rate': 0.01, 'max_bin': 4050, 'max_depth': 16, 'metric': 'auc', 'min_child_samples': 450, 'min_data_in_bin': 5700, 'n_estimators': 1750, 'n_jobs': 4, 'num_leaves': 61, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 10, 'reg_lambda': 0.001, 'subsample': 0.7, 'subsample_freq': 45},
{'boosting_type': 'gbdt', 'colsample_bytree': 0.3, 'learning_rate': 0.014, 'max_bin': 9950, 'max_depth': 26, 'metric': 'auc', 'min_child_samples': 1800, 'min_data_in_bin': 2150, 'n_estimators': 1650, 'n_jobs': 4, 'num_leaves': 151, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 0.1, 'reg_lambda': 0.001, 'subsample': 0.75, 'subsample_freq': 85},    
{'boosting_type': 'gbdt', 'colsample_bytree': 0.45, 'learning_rate': 0.019, 'max_bin': 7450, 'max_depth': 10, 'metric': 'auc', 'min_child_samples': 1450, 'min_data_in_bin': 1450, 'n_estimators': 1250, 'n_jobs': 4, 'num_leaves': 61, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 0.05, 'reg_lambda': 5, 'subsample': 0.9, 'subsample_freq': 55}, 
{'boosting_type': 'gbdt', 'colsample_bytree': 0.1, 'learning_rate': 0.03, 'max_bin': 8950, 'max_depth': 8, 'metric': 'auc', 'min_child_samples': 2550, 'min_data_in_bin': 4450, 'n_estimators': 1950, 'n_jobs': 4, 'num_leaves': 56, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 10, 'reg_lambda': 0.001, 'subsample': 0.75, 'subsample_freq': 13}, 
{'boosting_type': 'gbdt', 'colsample_bytree': 0.5, 'learning_rate': 0.026, 'max_bin': 3000, 'max_depth': 21, 'metric': 'auc', 'min_child_samples': 3200, 'min_data_in_bin': 600, 'n_estimators': 1950, 'n_jobs': 4, 'num_leaves': 71, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 5, 'reg_lambda': 1, 'subsample': 0.95, 'subsample_freq': 5}    
]

X = df[feats]
y = df[['target']]
skf = KFold(n_splits=5)

for params in binary_params:
    print(params)
    num_boost_round = params['n_estimators']
    params_ = copy.deepcopy(params)
    del params_['n_estimators']  
    
    losses = []
    
    for train_index, test_index in skf.split(X, y):
        
        #----start: data prep-------------------------------------
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        #----end: data prep-------------------------------------
        
        #-----start: train trees---------------------------------------
        tree_model = train_trees(X_train, y_train['target'].values,
                                 num_round=num_boost_round, 
                                 params=params_
                                )

        y_preds = tree_model.predict(X_test, num_iteration=tree_model.best_iteration)
#         loss = roc_auc_score(y_test, y_preds)
#         loss = f1_score(y_test['target'].values, (y_preds>=0.9).astype(int))
#         loss = amex_metric(y_test, 
#                            pd.DataFrame(data={'prediction': y_preds}))
        loss = amex_metric(y_test['target'].values, y_preds)
        print(loss)
        losses.append(loss)
    print('all:', np.mean(losses))

{'boosting_type': 'gbdt', 'colsample_bytree': 0.1, 'learning_rate': 0.03, 'max_bin': 8950, 'max_depth': 8, 'metric': 'auc', 'min_child_samples': 2550, 'min_data_in_bin': 4450, 'n_estimators': 1950, 'n_jobs': 4, 'num_leaves': 56, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 10, 'reg_lambda': 0.001, 'subsample': 0.75, 'subsample_freq': 13}
0.7920969675654579
0.7862520945403855
0.7907922726251617
0.7936374471393376
0.7923810177876274
all: 0.791031959931594
{'boosting_type': 'gbdt', 'colsample_bytree': 0.1, 'learning_rate': 0.02, 'max_bin': 8500, 'max_depth': 22, 'metric': 'auc', 'min_child_samples': 600, 'min_data_in_bin': 3400, 'n_estimators': 1100, 'n_jobs': 4, 'num_leaves': 236, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 0.0001, 'reg_lambda': 10, 'subsample': 0.7, 'subsample_freq': 73}
0.7916330688271154
0.7878642062573007
0.7886674418437842
0.7945399745629358
0.7932379990010137
all: 0.79118853809843
{'boosting_type': 'gbdt', 'colsample_bytree': 0.1, 'learni

In [22]:
del df, X, y, skf, tree_model, y_preds
gc.collect()

25

### train models

In [23]:
import lightgbm as lgb
# from lightgbm import LGBMClassifier

def train_trees(X_train, y_train, num_round=100, params={}, 
                verbosity=-1, 
                save_model = False, model_file = None):
    
    dtrain = lgb.Dataset(X_train, y_train)
    params['verbosity']=verbosity
    
    tree_model = lgb.train(params,
                dtrain,
                num_boost_round=num_round
                          )
    

    if save_model:
        if model_file is None:
            model_file = 'lgb.pkl'
        with open(model_file, 'wb') as fout:
            pickle.dump(tree_model, fout)
   
    del dtrain, tree_model
    gc.collect()

def inference_trees(X, model_files):
    
    pred_list=[]

    for model_file in model_files:
        with open(model_file, 'rb') as f_in:
            tree_model = pickle.load(f_in)
            y_preds = tree_model.predict(X, num_iteration=tree_model.best_iteration)
            
        pred_list.append(y_preds)
    
    
    return np.array(pred_list)

In [10]:
binary_params = [
{'boosting_type': 'gbdt', 'colsample_bytree': 0.1, 'learning_rate': 0.03, 'max_bin': 8950, 'max_depth': 8, 'metric': 'auc', 'min_child_samples': 2550, 'min_data_in_bin': 4450, 'n_estimators': 1950, 'n_jobs': 4, 'num_leaves': 56, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 10, 'reg_lambda': 0.001, 'subsample': 0.75, 'subsample_freq': 13}, 
{'boosting_type': 'gbdt', 'colsample_bytree': 0.1, 'learning_rate': 0.02, 'max_bin': 8500, 'max_depth': 22, 'metric': 'auc', 'min_child_samples': 600, 'min_data_in_bin': 3400, 'n_estimators': 1100, 'n_jobs': 4, 'num_leaves': 236, 'objective': 'binary', 'random_state': 1234, 'reg_alpha': 0.0001, 'reg_lambda': 10, 'subsample': 0.7, 'subsample_freq': 73}, 
]

model_files = [f'lgb{i+1}.pkl' for i in range(len(binary_params))]

print(model_files)


['lgb1.pkl', 'lgb2.pkl']


In [11]:
feats486 = ['D_48__max', 'D_129__mean', 'D_129__last', 'D_107__last__log', 'D_49__last__log', 'B_1__max', 'D_112__range', 'D_43__min', 'D_39__range', 'D_78__last', 'S_15__range', 'B_18__last', 'D_113__max', 'R_15__mean__log', 'B_12__last', 'B_23__last2max', 'S_24__mean', 'S_20__max', 'R_8__range', 'P_2__max', 'B_9__min', 'R_6__mean__log', 'D_70__range', 'R_4__mean', 'B_3__max', 'D_133__max', 'D_92__mean__log', 'B_18__last__log', 'D_92__max', 'P_4__first', 'D_74__last', 'B_28__min', 'D_121__first', 'D_131__max', 'D_79__mean', 'R_24__max', 'S_3__min', 'D_74__range', 'D_44__mean__log', 'B_4__mean', 'D_119__min', 'D_43__last__log', 'D_115__min', 'B_37__range', 'R_5__mean', 'D_130__min', 'D_71__last', 'R_13__mean2std', 'P_2__min', 'B_40__last', 'D_121__mean', 'B_28__mean', 'R_3__last', 'D_41__std', 'D_60__last', 'D_44__last__log', 'D_70__last', 'S_6__min', 'D_53__range', 'R_4__last__log', 'R_2__max', 'D_43__mean__log', 'R_4__mean__log', 'B_2__last', 'D_77__first', 'D_135__last__log', 'P_3__min', 'B_16__range', 'D_75__std', 'B_9__range', 'S_11__min', 'D_134__mean2std', 'B_4__last__log', 'D_55__range', 'R_1__range', 'B_2__last__log', 'D_48__std', 'S_6__max', 'D_120=1.0', 'D_64=U', 'D_44__last', 'B_11__mean__log', 'B_37__min', 'S_17__min', 'B_23__last', 'P_4__range', 'R_5__range', 'D_55__min', 'S_5__last', 'B_8__last__log', 'D_39__mean__log', 'S_23__mean', 'B_2__last2max', 'D_131__range', 'D_112__mean', 'B_19__min', 'B_19__first', 'D_137__last__log', 'S_11__std', 'D_84__last', 'B_8__mean__log', 'B_26__last__log', 'R_6__last__log', 'S_8__min', 'B_20__mean2std', 'B_7__last', 'D_61__last', 'S_3__std', 'D_45__last__log', 'B_30=1.0', 'D_62__mean', 'B_12__mean__log', 'R_10__mean__log', 'B_8__min', 'R_8__std', 'B_27__last__log', 'R_7__std', 'D_61__first', 'D_106__mean2std', 'B_4__range', 'R_7__mean__log', 'S_3__range', 'R_8__mean', 'B_40__last__log', 'D_78__first', 'D_58__last2max', 'B_1__range', 'R_1__last__log', 'D_42__range', 'D_81__range', 'D_130__max', 'S_23__last', 'R_15__mean', 'B_4__first', 'B_20__range', 'D_48__min', 'B_22__last__log', 'D_58__range', 'B_3__last2max', 'S_25__mean', 'B_26__last', 'D_44__first', 'B_19__mean', 'B_24__max', 'D_128__last__log', 'D_120__last', 'D_78__last__log', 'D_111__last', 'D_53__mean', 'D_52__first', 'D_43__max', 'B_20__last2max', 'D_45__max', 'D_80__last', 'D_44__std', 'D_74__last__log', 'D_127__last__log', 'D_77__range', 'D_58__last__log', 'B_5__mean', 'D_112__last2max', 'P_3__mean', 'B_38__last', 'B_23__std', 'B_1__last2max', 'D_89__mean', 'B_25__last2max', 'D_65__last__log', 'S_13__last2max', 'D_59__min', 'D_70__std', 'P_2__range', 'D_71__mean__log', 'D_39__max', 'S_25__min', 'D_44__max', 'D_46__last', 'R_1__max', 'D_107__mean__log', 'D_58__last', 'P_4__max', 'B_1__last', 'D_55__std', 'D_60__last__log', 'D_84__mean', 'D_92__last', 'B_10__max', 'D_135__last', 'P_3__last', 'D_93__last', 'B_9__last__log', 'R_20__std', 'B_6__min', 'D_41__min', 'S_8__mean', 'D_79__std', 'R_5__std', 'R_26__mean2std', 'D_61__min', 'S_7__first', 'D_112__min', 'R_6__last', 'R_7__last', 'R_21__max', 'B_3__last__log', 'D_39__std', 'D_68=6.0', 'R_1__mean', 'B_14__mean', 'B_11__range', 'D_39__mean', 'D_58__min', 'B_19__max', 'R_15__range', 'D_133__mean__log', 'B_28__max', 'B_19__last', 'B_11__last2max', 'S_7__max', 'D_65__mean__log', 'D_79__first', 'R_16__max', 'B_10__mean', 'S_15__last', 'B_19__last__log', 'B_8__max', 'D_130__mean', 'D_74__mean__log', 'B_4__last', 'D_51__first', 'D_42__min', 'R_5__last', 'R_22__max', 'B_21__mean__log', 'R_1__std', 'D_78__mean__log', 'S_25__first', 'P_2__first', 'D_51__mean', 'B_3__range', 'S_27__last', 'S_22__max', 'B_38__nunique', 'D_45__last', 'B_14__last', 'D_65__range', 'D_144__last', 'D_39__last', 'B_17__last', 'D_48__last', 'D_89__std', 'R_10__last', 'B_18__mean__log', 'R_1__last', 'R_10__last__log', 'D_54__range', 'B_16__last2max', 'S_23__max', 'R_27__min', 'B_2__mean__log', 'P_3__first', 'D_112__last__log', 'D_72__last', 'S_22__last', 'B_5__last__log', 'S_27__mean2std', 'D_50__last', 'D_51__max', 'S_24__max', 'B_18__min', 'R_27__max', 'D_54__min', 'D_113__last__log', 'D_43__range', 'D_55__max', 'D_52__mean', 'B_22__max', 'D_89__max', 'D_45__mean', 'R_13__range', 'R_14__mean__log', 'B_38=2.0', 'D_132__min', 'D_106__last', 'S_7__last', 'D_41__range', 'R_2__last2min', 'S_8__last', 'B_9__last2max', 'D_78__mean', 'B_6__last2max', 'R_4__max', 'D_41__last__log', 'R_21__std', 'D_72__max', 'B_9__mean', 'B_32__range', 'D_65__mean', 'B_23__max', 'D_51__std', 'B_24__mean__log', 'S_23__std', 'D_53__last__log', 'D_42__last', 'D_59__last', 'R_16__std', 'S_9__max', 'D_84__range', 'D_44__range', 'R_8__mean__log', 'P_2__last', 'D_115__last', 'D_45__first', 'D_59__max', 'D_130__mean__log', 'S_7__mean', 'R_1__mean__log', 'R_3__max', 'B_3__std', 'D_72__mean__log', 'D_75__min', 'R_17__range', 'B_32__last__log', 'P_2__std', 'R_8__max', 'D_84__std', 'B_17__min', 'R_21__mean', 'D_114__nunique', 'R_4__last', 'B_9__last', 'D_46__mean', 'D_77__min', 'D_75__last', 'B_9__mean__log', 'D_55__first', 'D_79__mean__log', 'B_4__max', 'B_10__first', 'D_47__min', 'B_24__last__log', 'B_37__mean', 'B_14__last2max', 'D_109__last', 'S_3__max', 'D_62__max', 'R_10__range', 'R_21__range', 'R_10__mean', 'D_79__last', 'B_37__std', 'B_5__mean__log', 'B_4__min', 'D_74__last2max', 'B_1__mean', 'D_78__max', 'S_23__range', 'D_84__last__log', 'D_42__first', 'B_37__max', 'B_18__first', 'D_55__mean', 'P_4__min', 'D_115__mean', 'S_24__last', 'D_102__last', 'D_41__mean', 'R_8__last', 'B_2__std', 'R_1__last2min', 'D_55__last__log', 'D_77__last', 'B_6__first', 'D_59__mean', 'R_15__std', 'B_28__last2max', 'B_7__last2max', 'B_22__last', 'B_29__last__log', 'D_47__max', 'B_25__min', 'D_64__last=O', 'R_7__mean', 'R_25__range', 'B_9__max', 'R_3__first', 'D_47__first', 'B_2__first', 'B_11__min', 'D_65__max', 'B_3__mean__log', 'D_54__mean', 'B_23__min', 'D_53__mean__log', 'D_84__max', 'B_16__min', 'D_131__last', 'B_16__mean', 'B_23__last__log', 'D_118__first', 'R_8__last__log', 'S_20__last', 'D_132__mean2std', 'D_122__mean', 'R_6__std', 'D_75__mean', 'B_22__mean', 'D_42__max', 'B_10__last', 'R_2__first', 'B_4__last2max', 'B_20__last', 'D_39__last__log', 'B_7__first', 'B_37__last', 'P_2__last2max', 'R_3__mean', 'B_20__last__log', 'R_13__std', 'B_16__first', 'P_2__mean', 'D_55__last', 'D_122__min', 'R_2__range', 'R_16__range', 'D_44__last2max', 'B_22__min', 'D_75__range', 'D_43__std', 'D_75__last__log', 'D_96__last', 'B_17__last__log', 'R_20__last', 'B_17__last2max', 'D_45__mean__log', 'D_138__last', 'B_30__nunique', 'D_43__last', 'D_47__mean', 'B_11__last', 'R_2__last', 'D_134__min', 'R_6__max', 'B_2__range', 'B_1__first', 'D_41__max', 'D_70__mean', 'D_120=0.0', 'S_25__std', 'D_77__mean', 'D_74__min', 'S_25__last', 'B_33__last', 'B_2__mean', 'D_102__max', 'B_33__last__log', 'D_125__last__log', 'D_114=0.0', 'B_16__max', 'B_25__last', 'R_20__mean__log', 'D_75__mean__log', 'R_10__first', 'S_3__last', 'B_26__mean__log', 'D_118__min', 'R_28__last__log', 'R_2__std', 'R_16__mean__log', 'P_4__std', 'D_65__std', 'D_51__mean__log', 'D_75__last2max', 'D_112__last', 'S_20__mean', 'D_131__mean__log', 'B_3__last', 'R_11__std', 'R_2__last__log', 'R_13__last', 'S_25__last2max', 'S_15__std', 'D_123__last__log', 'D_58__max', 'B_25__mean', 'B_1__min']
feats497 = ['D_48__max', 'D_107__last__log', 'D_49__last__log', 'B_1__max', 'D_112__range', 'D_43__min', 'D_39__range', 'D_78__last', 'B_42__last__log', 'B_18__last', 'R_15__mean__log', 'B_12__last', 'B_17__mean', 'R_13__mean__log', 'S_24__mean', 'R_17__mean__log', 'R_8__range', 'P_2__max', 'B_9__min', 'R_14__last__log', 'B_19__std', 'D_70__range', 'R_4__mean', 'B_30=2.0', 'B_3__max', 'D_92__mean__log', 'B_18__last__log', 'D_92__max', 'P_4__first', 'D_74__last', 'B_28__min', 'D_121__first', 'D_131__max', 'D_79__mean', 'R_24__max', 'S_3__min', 'D_74__range', 'D_44__mean__log', 'B_4__mean', 'R_13__max', 'S_15__max', 'S_8__last2max', 'D_119__min', 'D_43__last__log', 'D_115__min', 'R_2__mean__log', 'R_27__mean', 'B_37__range', 'S_6__last2max', 'R_5__mean', 'D_130__min', 'D_71__last', 'R_13__mean2std', 'P_2__min', 'B_40__last', 'D_52__min', 'R_3__last', 'D_41__std', 'D_60__last', 'D_120=999.0', 'D_44__last__log', 'D_53__range', 'D_131__last__log', 'R_4__last__log', 'R_2__max', 'D_43__mean__log', 'R_4__mean__log', 'B_2__last', 'D_135__last__log', 'P_3__min', 'B_16__range', 'B_11__max', 'S_11__min', 'B_4__last__log', 'D_55__range', 'R_1__range', 'B_2__last__log', 'D_48__std', 'S_6__max', 'D_53__mean2std', 'D_120=1.0', 'R_14__last', 'D_64=U', 'D_44__last', 'B_11__mean__log', 'B_37__min', 'S_17__min', 'B_23__last', 'B_5__last', 'D_48__mean', 'R_5__range', 'S_22__first', 'S_5__last', 'D_39__mean__log', 'S_23__mean', 'B_2__last2max', 'D_131__range', 'B_23__mean', 'D_112__mean', 'D_83__last', 'R_24__last', 'R_3__std', 'D_113__mean__log', 'B_19__min', 'S_11__std', 'D_84__last', 'B_8__mean__log', 'B_26__last__log', 'B_16__last', 'R_6__last__log', 'D_79__range', 'S_8__min', 'B_7__last', 'D_39__mean2std', 'D_61__last', 'D_45__last__log', 'B_30=1.0', 'D_62__mean', 'R_10__mean__log', 'B_8__min', 'R_8__std', 'D_120__nunique', 'B_27__last__log', 'R_7__std', 'R_12__mean', 'D_61__first', 'D_106__mean2std', 'B_4__range', 'D_81__mean__log', 'R_8__mean', 'S_3__range', 'B_40__last__log', 'D_78__first', 'D_58__last2max', 'B_1__range', 'D_124__last', 'R_1__last__log', 'R_2__min', 'D_130__max', 'S_23__last', 'R_15__mean', 'D_48__range', 'B_4__first', 'B_20__range', 'D_48__min', 'B_22__last__log', 'D_58__range', 'B_3__last2max', 'S_25__mean', 'B_26__last', 'D_44__first', 'B_19__mean', 'D_128__last__log', 'D_78__last__log', 'D_111__last', 'D_53__mean', 'D_43__max', 'B_20__last2max', 'D_45__max', 'D_44__std', 'D_74__last__log', 'D_127__last__log', 'D_77__range', 'B_5__mean', 'D_112__last2max', 'P_3__mean', 'B_38__last', 'B_23__std', 'B_1__last2max', 'B_25__last2max', 'D_65__last__log', 'S_13__last2max', 'D_59__min', 'D_70__std', 'P_2__range', 'D_39__max', 'S_25__min', 'D_44__max', 'D_46__last', 'R_1__max', 'S_23__mean2std', 'D_56__last', 'B_23__range', 'D_107__mean__log', 'D_58__last', 'P_4__max', 'B_1__last', 'D_55__std', 'D_60__last__log', 'D_84__mean', 'D_92__last', 'D_77__max', 'B_10__max', 'D_135__last', 'P_3__last', 'D_93__last', 'B_9__last__log', 'R_20__std', 'B_6__min', 'D_41__min', 'B_32__std', 'S_8__mean', 'D_79__std', 'S_6__last', 'R_5__std', 'R_26__mean2std', 'D_61__min', 'S_7__first', 'D_116__nunique', 'R_10__max', 'R_6__last', 'R_7__last', 'D_123__mean', 'R_21__max', 'B_3__last__log', 'D_39__std', 'D_68=6.0', 'R_1__mean', 'B_14__mean', 'R_10__std', 'B_18__last2max', 'B_3__mean', 'D_39__mean', 'B_19__max', 'B_7__min', 'D_133__mean__log', 'B_28__max', 'B_19__last', 'B_11__last2max', 'D_48__last2max', 'D_74__mean', 'D_108__last__log', 'S_7__max', 'R_16__max', 'B_10__mean', 'S_15__last', 'D_52__last', 'B_19__last__log', 'B_8__max', 'D_130__mean', 'D_74__mean__log', 'R_25__max', 'B_4__last', 'D_51__first', 'D_42__min', 'R_5__last', 'R_22__max', 'R_1__std', 'D_78__mean__log', 'S_25__first', 'P_2__first', 'B_21__last__log', 'D_51__mean', 'B_3__range', 'S_27__last', 'S_22__max', 'D_62__min', 'D_45__last', 'D_65__range', 'R_1__min', 'D_41__last', 'B_17__last', 'D_48__last', 'D_119__max', 'D_72__range', 'D_89__std', 'R_10__last', 'B_18__mean__log', 'R_1__last', 'D_70__first', 'B_10__last2max', 'R_10__last__log', 'R_2__mean', 'D_54__range', 'B_16__last2max', 'S_23__max', 'R_27__min', 'B_2__mean__log', 'P_3__first', 'B_33__max', 'D_112__last__log', 'S_6__std', 'D_72__last', 'S_22__last', 'S_27__mean2std', 'D_50__last', 'D_51__max', 'S_24__max', 'B_18__min', 'R_27__max', 'D_41__mean__log', 'D_43__range', 'D_55__max', 'D_52__mean', 'D_138__mean2std', 'D_89__max', 'R_13__range', 'B_38=2.0', 'S_12__max', 'D_132__min', 'D_106__last', 'B_33__mean__log', 'S_7__last', 'D_41__range', 'R_2__last2min', 'B_9__last2max', 'D_78__mean', 'B_6__last2max', 'R_4__max', 'D_41__last__log', 'D_72__max', 'B_9__mean', 'B_37__last2max', 'B_32__range', 'D_65__mean', 'B_38=4.0', 'D_51__std', 'B_24__mean__log', 'S_23__std', 'D_42__last', 'D_59__last', 'R_16__std', 'D_84__range', 'D_44__range', 'D_54__last2max', 'R_8__mean__log', 'B_8__mean', 'P_2__last', 'D_45__first', 'D_118__last__log', 'D_58__mean__log', 'R_1__mean__log', 'R_3__max', 'B_3__std', 'D_72__mean__log', 'D_75__min', 'R_17__range', 'B_32__last__log', 'P_2__std', 'R_8__max', 'D_84__std', 'B_17__min', 'D_61__last2max', 'R_27__first', 'R_21__mean', 'B_33__mean', 'D_44__mean', 'R_4__last', 'R_11__mean', 'B_9__last', 'D_46__mean', 'D_77__min', 'D_75__last', 'B_31__nunique', 'D_55__first', 'B_12__last__log', 'D_79__mean__log', 'B_4__max', 'B_10__first', 'D_47__min', 'B_24__last__log', 'B_37__mean', 'B_14__last2max', 'S_22__mean', 'S_3__max', 'D_62__max', 'D_118__last', 'R_10__range', 'R_10__mean', 'D_79__last', 'B_37__std', 'B_5__mean__log', 'D_112__std', 'B_1__mean', 'D_78__max', 'S_23__range', 'B_17__mean__log', 'D_84__last__log', 'D_42__first', 'B_37__max', 'B_18__first', 'D_55__mean', 'D_48__first', 'P_4__min', 'D_115__mean', 'S_24__last', 'D_102__last', 'D_41__mean', 'R_8__last', 'D_55__last__log', 'D_77__last', 'B_6__first', 'R_15__std', 'B_28__last2max', 'B_7__last2max', 'B_22__last', 'B_29__last__log', 'D_47__max', 'B_25__min', 'D_74__max', 'D_69__min', 'R_25__range', 'B_9__max', 'R_3__first', 'D_47__first', 'B_2__first', 'B_11__min', 'D_65__max', 'R_17__std', 'B_3__mean__log', 'D_54__mean', 'B_23__min', 'D_53__mean__log', 'D_84__max', 'B_16__min', 'B_42__last', 'R_3__range', 'D_131__last', 'B_16__mean', 'B_23__last__log', 'D_118__first', 'R_8__last__log', 'S_16__last', 'R_6__std', 'D_75__mean', 'D_42__max', 'B_11__mean', 'B_4__last2max', 'B_20__last', 'D_39__last__log', 'B_37__last', 'P_2__last2max', 'R_3__mean', 'B_20__last__log', 'R_13__std', 'B_16__first', 'P_2__mean', 'D_55__last', 'D_122__min', 'D_116=999.0', 'R_16__range', 'D_44__last2max', 'D_75__range', 'D_43__std', 'D_75__last__log', 'B_17__last__log', 'B_17__last2max', 'D_55__mean__log', 'D_45__mean__log', 'D_138__last', 'D_43__last', 'D_47__mean', 'B_11__last', 'D_134__min', 'R_6__max', 'B_2__range', 'B_1__first', 'D_41__max', 'D_70__mean', 'D_120=0.0', 'S_25__std', 'D_77__mean', 'D_137__last', 'D_74__min', 'S_25__last', 'B_33__last', 'R_5__max', 'B_2__mean', 'B_33__last__log', 'D_121__last', 'D_125__last__log', 'B_25__last', 'D_75__mean__log', 'R_10__first', 'S_3__last', 'B_26__mean__log', 'D_118__min', 'R_28__last__log', 'R_2__std', 'R_16__mean__log', 'P_4__std', 'D_65__std', 'D_51__mean__log', 'D_75__last2max', 'D_112__last', 'S_20__mean', 'D_131__mean__log', 'B_3__last', 'S_20__range', 'R_2__last__log', 'D_131__std', 'R_13__last', 'S_25__last2max', 'S_15__std', 'D_123__last__log', 'D_58__max', 'B_25__mean']
feats641 = ['B_10__last', 'B_10__last2max', 'B_10__mean', 'B_10__min', 'B_11__last', 'B_11__last2max', 'B_11__last__log', 'B_12__last', 'B_12__last__log', 'B_12__mean__log', 'B_13__last', 'B_14__last', 'B_14__last2max', 'B_14__min', 'B_15__last', 'B_15__mean', 'B_16__first', 'B_16__range', 'B_16__std', 'B_17__last', 'B_17__last2max', 'B_17__mean', 'B_17__mean__log', 'B_17__min', 'B_18__last2max', 'B_18__min', 'B_19__last2max', 'B_19__range', 'B_19__std', 'B_1__last', 'B_1__last2max', 'B_1__max', 'B_1__std', 'B_20__last', 'B_20__last2max', 'B_20__mean', 'B_21__last', 'B_21__last__log', 'B_21__max', 'B_21__mean__log', 'B_22__first', 'B_22__last', 'B_22__last__log', 'B_22__max', 'B_22__min', 'B_22__range', 'B_23__last', 'B_23__last2max', 'B_23__last__log', 'B_23__range', 'B_23__std', 'B_24__last', 'B_24__last__log', 'B_24__max', 'B_24__mean', 'B_24__mean__log', 'B_24__std', 'B_25__last', 'B_25__last2max', 'B_25__min', 'B_26__last', 'B_26__last__log', 'B_26__mean__log', 'B_27__last', 'B_28__first', 'B_28__last2max', 'B_28__mean', 'B_28__min', 'B_2__last', 'B_2__last2max', 'B_2__last__log', 'B_2__mean', 'B_2__mean__log', 'B_2__min', 'B_32__last', 'B_32__last__log', 'B_32__max', 'B_32__mean__log', 'B_32__range', 'B_32__std', 'B_33__first', 'B_33__last', 'B_36__last', 'B_36__last__log', 'B_37__last', 'B_37__last2max', 'B_37__max', 'B_37__min', 'B_37__std', 'B_3__last', 'B_3__last2max', 'B_3__last__log', 'B_3__max', 'B_3__range', 'B_3__std', 'B_40__last2max', 'B_40__max', 'B_40__mean__log', 'B_40__min', 'B_41__last', 'B_41__last__log', 'B_4__last', 'B_4__last2max', 'B_4__last__log', 'B_4__max', 'B_4__mean__log', 'B_4__min', 'B_4__range', 'B_4__std', 'B_5__last', 'B_5__last__log', 'B_5__mean', 'B_5__mean__log', 'B_6__last2max', 'B_6__min', 'B_7__last', 'B_7__last2max', 'B_7__max', 'B_7__range', 'B_7__std', 'B_8__first', 'B_8__last', 'B_8__last__log', 'B_8__max', 'B_8__mean', 'B_8__mean__log', 'B_8__min', 'B_9__last', 'B_9__last2max', 'B_9__last__log', 'B_9__max', 'B_9__mean', 'B_9__mean__log', 'B_9__min', 'B_9__std', 'D_102__max', 'D_103__last', 'D_104__last', 'D_105__max', 'D_107__last', 'D_107__last__log', 'D_107__max', 'D_107__mean__log', 'D_109__last', 'D_109__last__log', 'D_109__mean', 'D_112__last', 'D_112__last2max', 'D_112__last__log', 'D_112__mean', 'D_112__mean__log', 'D_112__min', 'D_112__range', 'D_112__std', 'D_113__last', 'D_113__max', 'D_115__first', 'D_115__min', 'D_118__first', 'D_118__last__log', 'D_118__mean__log', 'D_118__min', 'D_119__first', 'D_119__last', 'D_119__min', 'D_121__first', 'D_121__last', 'D_121__max', 'D_121__mean', 'D_121__min', 'D_122__first', 'D_122__last', 'D_122__max', 'D_122__mean', 'D_122__mean__log', 'D_122__min', 'D_123__last__log', 'D_123__mean', 'D_124__last', 'D_125__last', 'D_125__last__log', 'D_127__first', 'D_127__last', 'D_127__last__log', 'D_127__max', 'D_127__mean', 'D_127__mean__log', 'D_127__range', 'D_127__std', 'D_128__first', 'D_128__last', 'D_128__last__log', 'D_128__max', 'D_128__mean', 'D_128__mean__log', 'D_128__min', 'D_129__last', 'D_129__max', 'D_129__mean', 'D_130__first', 'D_130__last', 'D_130__max', 'D_130__mean', 'D_130__min', 'D_131__first', 'D_131__last', 'D_131__last__log', 'D_131__max', 'D_131__mean', 'D_131__min', 'D_131__range', 'D_131__std', 'D_133__last', 'D_133__last__log', 'D_133__max', 'D_133__mean__log', 'D_133__min', 'D_139__last', 'D_139__mean', 'D_140__last', 'D_140__last__log', 'D_140__max', 'D_141__last', 'D_143__last', 'D_144__last', 'D_145__last', 'D_39__last', 'D_39__last__log', 'D_39__max', 'D_39__mean', 'D_39__mean__log', 'D_39__range', 'D_39__std', 'D_41__last', 'D_41__last__log', 'D_41__max', 'D_41__mean', 'D_41__mean__log', 'D_41__min', 'D_41__range', 'D_41__std', 'D_42__first', 'D_42__last', 'D_42__max', 'D_42__mean', 'D_42__min', 'D_43__first', 'D_43__last', 'D_43__last__log', 'D_43__max', 'D_43__mean', 'D_43__mean__log', 'D_43__min', 'D_43__range', 'D_43__std', 'D_44__first', 'D_44__last', 'D_44__last2max', 'D_44__last__log', 'D_44__max', 'D_44__range', 'D_44__std', 'D_45__first', 'D_45__last', 'D_45__last__log', 'D_45__max', 'D_45__mean', 'D_45__mean__log', 'D_45__min', 'D_46__last', 'D_46__mean', 'D_46__min', 'D_47__first', 'D_47__last', 'D_47__max', 'D_47__mean', 'D_47__min', 'D_48__first', 'D_48__last', 'D_48__last2max', 'D_48__max', 'D_48__mean', 'D_48__min', 'D_48__range', 'D_48__std', 'D_49__last', 'D_49__last__log', 'D_50__last', 'D_51__first', 'D_51__last', 'D_51__last__log', 'D_51__max', 'D_51__mean', 'D_51__mean__log', 'D_51__min', 'D_51__range', 'D_51__std', 'D_52__first', 'D_52__last', 'D_52__max', 'D_52__mean', 'D_52__min', 'D_54__last', 'D_54__last2max', 'D_54__mean', 'D_54__min', 'D_54__range', 'D_54__std', 'D_55__first', 'D_55__last2max', 'D_55__max', 'D_55__min', 'D_55__range', 'D_55__std', 'D_56__min', 'D_58__range', 'D_59__first', 'D_59__last', 'D_59__max', 'D_59__mean', 'D_59__min', 'D_60__last', 'D_60__last__log', 'D_60__max', 'D_61__first', 'D_61__last', 'D_61__last2max', 'D_61__max', 'D_61__mean', 'D_61__min', 'D_62__first', 'D_62__last', 'D_62__last2max', 'D_62__max', 'D_62__mean', 'D_62__min', 'D_65__last', 'D_65__last__log', 'D_65__max', 'D_65__mean', 'D_65__mean__log', 'D_65__range', 'D_65__std', 'D_69__last', 'D_69__min', 'D_70__first', 'D_70__last', 'D_70__last__log', 'D_70__max', 'D_70__mean__log', 'D_70__min', 'D_70__std', 'D_71__last', 'D_71__last__log', 'D_71__mean__log', 'D_72__last', 'D_72__last__log', 'D_72__max', 'D_72__mean', 'D_72__mean__log', 'D_72__min', 'D_72__range', 'D_72__std', 'D_74__first', 'D_74__range', 'D_74__std', 'D_75__range', 'D_75__std', 'D_77__max', 'D_77__mean', 'D_77__min', 'D_78__first', 'D_78__last', 'D_78__last__log', 'D_78__mean__log', 'D_78__range', 'D_79__first', 'D_79__last', 'D_79__last__log', 'D_79__max', 'D_79__mean', 'D_79__mean__log', 'D_79__min', 'D_79__range', 'D_79__std', 'D_80__last', 'D_81__last', 'D_81__last__log', 'D_81__max', 'D_81__mean', 'D_81__std', 'D_83__last', 'D_84__last', 'D_84__last__log', 'D_84__max', 'D_84__mean', 'D_86__last', 'D_89__last', 'D_89__max', 'D_89__mean', 'D_89__range', 'D_89__std', 'D_91__last', 'D_91__max', 'D_91__mean__log', 'D_92__last', 'D_92__max', 'D_92__mean', 'D_92__mean__log', 'D_93__last', 'D_94__last', 'D_96__last', 'P_2__first', 'P_2__last', 'P_2__last2max', 'P_2__max', 'P_2__mean', 'P_2__min', 'P_2__range', 'P_2__std', 'P_3__first', 'P_3__last', 'P_3__last2max', 'P_3__max', 'P_3__mean', 'P_3__min', 'P_3__range', 'P_3__std', 'P_4__first', 'P_4__last__log', 'P_4__mean', 'P_4__min', 'P_4__range', 'P_4__std', 'R_10__first', 'R_10__last', 'R_10__last__log', 'R_10__max', 'R_10__mean__log', 'R_10__std', 'R_11__last', 'R_11__max', 'R_11__mean', 'R_11__mean__log', 'R_11__std', 'R_12__last', 'R_12__max', 'R_12__mean', 'R_12__min', 'R_12__range', 'R_12__std', 'R_13__last', 'R_13__max', 'R_13__mean', 'R_13__range', 'R_13__std', 'R_14__last', 'R_14__last__log', 'R_14__mean__log', 'R_15__last', 'R_15__max', 'R_15__mean', 'R_15__mean__log', 'R_15__range', 'R_15__std', 'R_16__last', 'R_16__max', 'R_16__mean', 'R_16__mean__log', 'R_16__std', 'R_17__last', 'R_17__max', 'R_17__mean__log', 'R_17__range', 'R_18__last', 'R_18__max', 'R_19__last', 'R_19__min', 'R_1__first', 'R_1__last', 'R_1__last2min', 'R_1__last__log', 'R_1__max', 'R_1__mean', 'R_1__mean__log', 'R_1__min', 'R_1__std', 'R_20__last', 'R_20__max', 'R_20__mean', 'R_20__mean__log', 'R_20__std', 'R_21__last', 'R_21__max', 'R_21__mean', 'R_21__mean__log', 'R_21__range', 'R_21__std', 'R_22__last', 'R_22__max', 'R_22__range', 'R_23__last', 'R_24__last', 'R_24__max', 'R_24__mean__log', 'R_24__range', 'R_24__std', 'R_25__last', 'R_25__max', 'R_25__range', 'R_25__std', 'R_27__first', 'R_27__last', 'R_27__max', 'R_27__mean', 'R_27__min', 'R_28__last', 'R_28__last__log', 'R_2__first', 'R_2__last', 'R_2__last2min', 'R_2__last__log', 'R_2__max', 'R_2__mean', 'R_2__mean__log', 'R_2__min', 'R_2__range', 'R_2__std', 'R_3__first', 'R_3__last', 'R_3__last__log', 'R_3__max', 'R_3__mean', 'R_3__mean__log', 'R_3__min', 'R_3__range', 'R_3__std', 'R_4__last', 'R_4__last__log', 'R_4__max', 'R_4__min', 'R_4__std', 'R_5__last', 'R_5__last__log', 'R_5__max', 'R_5__mean__log', 'R_5__std', 'R_6__last', 'R_6__last__log', 'R_6__max', 'R_6__mean__log', 'R_7__last', 'R_7__last__log', 'R_7__max', 'R_7__mean', 'R_7__range', 'R_7__std', 'R_8__last', 'R_8__last__log', 'R_8__max', 'R_8__mean', 'R_8__mean__log', 'R_8__std', 'S_11__last', 'S_11__mean', 'S_11__min', 'S_11__std', 'S_12__last', 'S_12__max', 'S_13__last', 'S_13__last2max', 'S_13__min', 'S_15__first', 'S_15__last', 'S_15__max', 'S_15__mean', 'S_15__std', 'S_16__last', 'S_16__mean__log', 'S_17__last', 'S_17__min', 'S_18__last', 'S_19__last', 'S_20__last', 'S_20__max', 'S_20__mean', 'S_20__mean__log', 'S_20__range', 'S_20__std', 'S_22__first', 'S_22__last', 'S_22__max', 'S_22__mean', 'S_23__last', 'S_23__max', 'S_23__mean', 'S_23__range', 'S_23__std', 'S_24__last', 'S_24__max', 'S_24__mean', 'S_25__first', 'S_25__last', 'S_25__last2max', 'S_25__mean', 'S_25__min', 'S_25__range', 'S_25__std', 'S_26__last', 'S_26__last__log', 'S_27__last', 'S_27__last2max', 'S_27__last__log', 'S_27__std', 'S_3__first', 'S_3__last', 'S_3__max', 'S_3__mean', 'S_3__min', 'S_3__range', 'S_3__std', 'S_5__last', 'S_5__last__log', 'S_5__mean__log', 'S_6__last', 'S_6__max', 'S_6__min', 'S_6__range', 'S_7__first', 'S_7__last', 'S_7__max', 'S_7__mean', 'S_7__min', 'S_7__range', 'S_7__std', 'S_8__last', 'S_8__last2max', 'S_8__mean', 'S_8__min', 'S_9__last', 'S_9__max']
feats372 = ['R_3__last__log', 'D_129__mean', 'D_129__last', 'D_107__last__log', 'D_113__last', 'D_49__last__log', 'D_43__min', 'D_78__last', 'D_39__range', 'D_52__max', 'B_42__last__log', 'B_2__max', 'D_113__max', 'R_13__mean__log', 'R_17__mean__log', 'P_2__max', 'D_76__last', 'D_49__mean2std', 'B_19__std', 'D_70__range', 'D_131__mean', 'B_3__max', 'P_4__first', 'D_131__max', 'D_74__range', 'D_44__mean__log', 'S_11__mean', 'B_4__mean', 'S_8__last2max', 'D_43__last__log', 'R_2__mean__log', 'S_20__std', 'B_37__range', 'R_5__mean', 'D_130__min', 'P_2__min', 'D_52__min', 'B_2__min', 'D_79__last__log', 'D_121__mean', 'B_21__last', 'S_16__mean__log', 'R_3__last', 'D_60__last', 'D_120=999.0', 'D_44__last__log', 'D_43__mean__log', 'R_4__mean__log', 'D_77__first', 'B_2__last', 'D_47__last', 'P_3__min', 'D_75__std', 'B_4__last__log', 'R_1__range', 'B_2__last__log', 'S_26__last', 'R_20__max', 'D_44__last', 'B_37__min', 'B_5__last', 'B_23__last', 'S_17__min', 'P_4__range', 'D_48__mean', 'D_55__min', 'R_12__last', 'D_62__last2max', 'D_39__mean__log', 'R_21__mean__log', 'B_23__mean', 'R_11__mean__log', 'R_3__std', 'B_19__first', 'P_3__last2max', 'D_84__last', 'B_26__last__log', 'R_6__last__log', 'D_79__range', 'B_7__last', 'D_39__mean2std', 'S_5__mean__log', 'D_108__last', 'S_3__std', 'D_45__last__log', 'D_62__mean', 'B_12__mean__log', 'R_7__std', 'D_61__first', 'B_4__range', 'R_1__last__log', 'D_42__range', 'S_23__last', 'B_4__first', 'B_20__range', 'B_22__last__log', 'D_58__range', 'B_3__last2max', 'S_25__mean', 'B_26__last', 'D_46__mean2std', 'R_26__last', 'D_128__last__log', 'D_54__std', 'D_53__mean', 'D_52__first', 'D_43__max', 'D_45__max', 'B_9__std', 'S_3__first', 'D_44__std', 'B_5__mean', 'D_58__last__log', 'B_15__mean', 'P_3__mean', 'B_38__last', 'R_17__last', 'B_1__last2max', 'B_25__last2max', 'D_53__last', 'D_59__min', 'D_81__last__log', 'B_11__last__log', 'R_3__mean__log', 'P_2__range', 'D_39__max', 'D_44__max', 'D_46__last', 'S_12__last', 'D_74__std', 'D_56__last', 'D_107__mean__log', 'D_58__last', 'B_1__last', 'D_60__last__log', 'D_84__mean', 'D_77__max', 'D_132__last', 'S_7__std', 'D_81__std', 'P_3__range', 'B_28__last', 'P_3__last', 'B_36__last', 'S_18__last', 'D_46__min', 'D_117=999.0', 'B_9__last__log', 'B_20__first', 'D_79__max', 'B_21__max', 'S_6__last', 'R_6__range', 'D_116__nunique', 'D_68__last', 'B_7__max', 'B_3__last__log', 'D_39__std', 'D_122__last', 'D_68=6.0', 'R_1__mean', 'R_10__std', 'B_14__mean', 'B_18__last2max', 'B_3__mean', 'D_39__mean', 'R_11__range', 'B_19__max', 'R_15__range', 'B_7__min', 'B_20__min', 'B_28__max', 'D_48__last2max', 'B_11__last2max', 'B_19__last', 'D_74__mean', 'D_114__last', 'D_72__std', 'S_7__max', 'D_52__last', 'S_15__last', 'D_143__last', 'B_4__last', 'D_51__first', 'B_33__min', 'D_42__min', 'B_21__mean__log', 'D_78__mean__log', 'P_2__first', 'B_21__last__log', 'D_51__mean', 'D_127__first', 'B_3__range', 'D_62__min', 'D_45__last', 'B_14__last', 'D_65__range', 'D_122__max', 'S_17__last', 'D_39__last', 'D_41__last', 'D_48__last', 'B_6__mean', 'R_1__first', 'B_10__last2max', 'R_2__mean', 'D_54__range', 'D_77__mean__log', 'B_16__last2max', 'S_23__max', 'R_27__min', 'P_3__first', 'D_112__last__log', 'S_6__std', 'S_22__last', 'B_5__last__log', 'D_51__max', 'S_24__max', 'D_49__last', 'R_27__max', 'D_41__mean__log', 'D_54__min', 'D_55__max', 'B_22__max', 'R_12__min', 'S_25__range', 'B_4__mean__log', 'D_132__min', 'S_7__last', 'D_41__range', 'D_114=1.0', 'R_2__last2min', 'S_8__last', 'D_41__last__log', 'B_37__last2max', 'D_65__mean', 'D_45__min', 'D_74__first', 'B_4__std', 'B_22__std', 'D_42__last', 'S_23__std', 'D_54__last', 'D_59__last', 'D_44__range', 'D_54__last2max', 'P_2__last', 'D_58__std', 'D_115__last', 'D_45__first', 'D_58__mean__log', 'R_1__mean__log', 'R_3__max', 'R_17__range', 'D_91__last', 'S_27__last2max', 'P_2__std', 'D_84__std', 'D_112__mean__log', 'S_3__mean', 'B_24__std', 'R_4__last', 'B_9__last', 'D_46__mean', 'S_9__last', 'D_133__min', 'D_64__last=U', 'D_75__last', 'B_9__mean__log', 'D_55__first', 'B_4__max', 'D_79__mean__log', 'B_10__first', 'B_6__last', 'B_37__mean', 'S_3__max', 'D_42__mean', 'D_79__last', 'D_74__last2max', 'B_1__mean', 'D_78__max', 'D_42__first', 'B_37__max', 'B_18__first', 'D_51__last', 'B_16__std', 'D_55__mean', 'P_4__min', 'S_24__last', 'D_41__mean', 'B_2__std', 'B_13__last', 'B_28__last2max', 'B_7__last2max', 'B_22__last', 'D_47__max', 'D_64__last=O', 'B_16__mean__log', 'R_25__range', 'B_9__max', 'R_3__first', 'D_47__first', 'B_2__first', 'P_4__mean', 'B_3__min', 'D_84__max', 'B_11__first', 'D_119__mean__log', 'R_3__range', 'B_23__last__log', 'D_43__mean', 'D_122__mean', 'D_129__max', 'B_22__mean', 'S_19__last', 'D_42__max', 'B_10__last', 'S_13__min', 'B_11__mean', 'B_4__last2max', 'D_39__last__log', 'B_37__last', 'D_43__first', 'P_2__last2max', 'R_3__mean', 'R_13__std', 'B_16__first', 'P_2__mean', 'D_110__last', 'D_116=999.0', 'D_44__last2max', 'D_75__range', 'D_43__std', 'D_96__last', 'B_17__last__log', 'D_55__mean__log', 'B_17__last2max', 'D_140__max', 'D_45__mean__log', 'D_43__last', 'D_47__mean', 'B_11__last', 'D_131__first', 'S_13__last', 'R_2__last', 'B_19__mean__log', 'D_137__last', 'R_5__max', 'D_102__max', 'D_121__last', 'D_114=0.0', 'B_16__max', 'R_20__mean__log', 'D_75__mean__log', 'B_26__mean__log', 'D_62__range', 'R_16__mean__log', 'D_65__std', 'D_51__mean__log', 'D_112__last', 'B_3__last', 'D_56__mean2std', 'D_137__mean2std', 'R_2__last__log', 'R_11__last', 'D_131__std', 'S_25__last2max', 'P_4__last__log', 'D_58__max', 'D_130__last__log', 'B_25__mean', 'B_1__min']
feats394 = ['D_129__mean', 'D_49__last__log', 'B_1__max', 'D_112__range', 'D_39__range', 'D_52__max', 'S_15__range', 'D_113__max', 'B_12__last', 'B_17__mean', 'D_78__std', 'P_2__max', 'D_76__last', 'D_49__mean2std', 'R_14__last__log', 'D_70__range', 'D_131__mean', 'B_30=2.0', 'B_3__max', 'D_74__last', 'B_28__min', 'D_79__mean', 'B_33__first', 'D_44__mean__log', 'S_11__mean', 'B_4__mean', 'S_15__max', 'D_43__last__log', 'D_115__min', 'R_2__mean__log', 'R_5__mean', 'D_130__min', 'D_71__last', 'R_13__mean2std', 'P_2__min', 'B_2__min', 'D_79__last__log', 'D_121__mean', 'B_21__last', 'R_3__last', 'D_41__std', 'D_60__last', 'R_24__std', 'D_44__last__log', 'D_131__last__log', 'R_2__max', 'D_43__mean__log', 'D_69__last', 'D_77__first', 'B_2__last', 'D_47__last', 'P_3__min', 'B_11__max', 'D_75__std', 'S_11__min', 'B_4__last__log', 'R_9__last', 'D_55__range', 'B_2__last__log', 'R_9__mean2std', 'D_120=1.0', 'R_20__max', 'D_44__last', 'B_37__min', 'B_5__last', 'B_23__last', 'D_89__mean__log', 'S_17__min', 'D_48__mean', 'R_5__range', 'D_55__min', 'D_87__last', 'D_42__mean2std', 'S_5__last', 'D_39__mean__log', 'R_21__mean__log', 'D_131__range', 'D_112__mean', 'R_11__mean__log', 'R_3__std', 'B_19__first', 'P_3__last2max', 'D_84__last', 'B_8__mean__log', 'B_26__last__log', 'R_6__last__log', 'B_40__min', 'D_79__range', 'B_7__last', 'D_39__mean2std', 'S_5__mean__log', 'D_108__last', 'D_45__last__log', 'D_62__mean', 'B_12__mean__log', 'B_8__min', 'B_27__last__log', 'D_68=1.0', 'B_4__range', 'D_84__mean__log', 'B_40__last__log', 'D_82__last', 'D_124__last', 'R_1__last__log', 'R_12__std', 'D_127__std', 'S_23__last', 'B_20__range', 'B_22__last__log', 'D_58__range', 'D_128__max', 'D_44__first', 'B_26__last', 'S_25__mean', 'R_26__last', 'B_24__max', 'D_78__last__log', 'D_54__std', 'R_18__max', 'D_52__first', 'D_43__max', 'D_53__mean', 'D_45__max', 'S_3__first', 'R_19__min', 'D_127__last__log', 'D_70__mean2std', 'B_5__mean', 'D_58__last__log', 'D_112__last2max', 'P_3__mean', 'B_15__mean', 'B_38__last', 'B_1__last2max', 'B_25__last2max', 'D_51__last__log', 'D_53__last', 'D_59__min', 'S_13__last2max', 'B_11__last__log', 'B_8__last', 'R_3__mean__log', 'P_2__range', 'D_39__max', 'S_25__min', 'D_46__last', 'D_56__last', 'D_107__mean__log', 'D_58__last', 'B_1__last', 'D_55__std', 'D_84__mean', 'D_77__max', 'D_132__last', 'S_7__std', 'D_130__last', 'B_28__last', 'P_3__last', 'B_36__last', 'S_18__last', 'D_92__mean', 'D_46__min', 'D_117=999.0', 'B_9__last__log', 'D_79__max', 'D_113__mean', 'S_6__last', 'S_8__mean', 'D_79__std', 'R_26__mean2std', 'R_6__range', 'D_68__last', 'D_128__mean', 'D_39__std', 'D_122__last', 'R_3__min', 'R_1__mean', 'D_68=6.0', 'R_10__std', 'B_11__range', 'S_15__first', 'D_39__mean', 'B_19__max', 'R_15__range', 'B_7__min', 'D_48__last2max', 'B_11__last2max', 'D_74__mean', 'D_114__last', 'D_72__std', 'S_7__max', 'D_65__mean__log', 'R_16__max', 'D_143__last', 'B_4__last', 'R_5__last', 'R_22__max', 'R_1__std', 'D_91__mean__log', 'R_11__max', 'D_109__last__log', 'P_2__first', 'B_21__last__log', 'D_51__mean', 'D_127__first', 'B_3__range', 'S_22__max', 'D_62__min', 'D_45__last', 'B_14__last', 'D_65__range', 'S_17__last', 'D_39__last', 'D_41__last', 'D_48__last', 'B_18__mean__log', 'B_10__last2max', 'R_10__last__log', 'R_2__mean', 'D_54__range', 'D_77__mean__log', 'S_23__max', 'S_6__std', 'S_22__last', 'B_5__last__log', 'S_27__mean2std', 'D_50__last', 'D_51__max', 'S_24__max', 'D_49__last', 'B_18__min', 'R_27__max', 'D_41__mean__log', 'D_54__min', 'D_113__last__log', 'D_43__range', 'D_125__last', 'D_55__max', 'D_138__mean2std', 'D_89__max', 'D_45__mean', 'R_12__min', 'S_25__range', 'B_4__mean__log', 'D_132__min', 'D_81__last', 'B_33__mean__log', 'S_6__range', 'D_41__range', 'B_30=0.0', 'D_114=1.0', 'D_56__min', 'R_2__last2min', 'S_8__last', 'D_133__last', 'D_41__last__log', 'S_11__last', 'D_65__mean', 'D_74__first', 'B_4__std', 'D_51__std', 'D_42__last', 'D_53__last__log', 'S_23__std', 'D_54__last', 'S_9__max', 'D_54__last2max', 'P_2__last', 'D_58__std', 'D_115__last', 'D_45__first', 'D_59__max', 'D_118__last__log', 'D_130__mean__log', 'S_7__mean', 'R_1__mean__log', 'R_3__max', 'D_107__last', 'D_75__min', 'R_17__range', 'D_91__last', 'D_84__std', 'D_112__mean__log', 'S_3__mean', 'R_21__mean', 'D_114__nunique', 'B_33__mean', 'D_44__mean', 'D_122__mean__log', 'R_11__mean', 'B_9__last', 'D_46__mean', 'S_9__last', 'D_70__max', 'B_38=1.0', 'B_12__last__log', 'B_4__max', 'D_79__mean__log', 'B_15__last', 'B_24__last__log', 'D_44__min', 'B_37__mean', 'S_3__max', 'D_62__max', 'D_42__mean', 'B_1__mean', 'D_84__last__log', 'R_16__mean', 'B_14__min', 'D_42__first', 'D_41__mean2std', 'D_55__mean', 'D_121__max', 'P_4__min', 'S_24__last', 'D_41__mean', 'R_8__last', 'B_2__std', 'R_1__last2min', 'D_55__last__log', 'S_15__mean', 'B_7__last2max', 'B_22__last', 'D_74__max', 'D_69__min', 'R_3__first', 'B_2__first', 'P_4__mean', 'D_53__mean__log', 'D_84__max', 'D_106__last__log', 'R_3__range', 'B_23__last__log', 'B_40__last2max', 'D_129__max', 'R_6__std', 'B_22__mean', 'D_42__max', 'S_13__min', 'R_5__last__log', 'B_4__last2max', 'B_20__last', 'D_39__last__log', 'B_37__last', 'P_2__last2max', 'R_3__mean', 'B_20__last__log', 'B_16__first', 'P_2__mean', 'D_119__last__log', 'R_16__range', 'D_44__last2max', 'D_75__range', 'D_43__std', 'R_20__last', 'D_55__mean__log', 'B_17__last2max', 'D_140__max', 'D_45__mean__log', 'B_11__last', 'B_38=7.0', 'D_131__first', 'S_13__last', 'R_2__last', 'R_7__max', 'R_24__mean', 'B_19__mean__log', 'D_41__max', 'D_120=0.0', 'S_25__std', 'D_137__last', 'S_25__last', 'B_2__mean', 'B_33__last__log', 'D_121__last', 'B_24__last', 'D_114=0.0', 'B_25__last', 'D_139__mean', 'R_20__mean__log', 'S_3__last', 'D_115__last__log', 'R_16__mean__log', 'D_51__mean__log', 'D_112__last', 'B_3__last', 'D_56__mean2std', 'R_11__std', 'D_62__first', 'R_2__last__log', 'R_11__last', 'D_131__min', 'S_25__last2max', 'P_4__last__log']
feats533 = ['D_129__mean', 'D_129__last', 'D_107__last__log', 'D_49__last__log', 'B_1__max', 'D_112__range', 'D_43__min', 'D_78__last', 'D_39__range', 'D_52__max', 'S_15__range', 'B_2__max', 'D_113__max', 'B_12__last', 'B_17__mean', 'D_78__std', 'P_2__max', 'D_76__last', 'D_49__mean2std', 'B_9__min', 'R_6__mean__log', 'R_14__last__log', 'B_19__std', 'D_70__range', 'D_131__mean', 'B_30=2.0', 'B_3__max', 'D_92__mean__log', 'P_4__first', 'D_74__last', 'B_28__min', 'D_79__mean', 'B_33__first', 'S_3__min', 'D_74__range', 'D_44__mean__log', 'S_11__mean', 'B_4__mean', 'S_15__max', 'S_8__last2max', 'D_43__last__log', 'D_115__min', 'R_2__mean__log', 'D_71__last__log', 'S_20__std', 'R_5__mean', 'D_130__min', 'D_71__last', 'R_13__mean2std', 'P_2__min', 'B_2__min', 'D_79__last__log', 'D_121__mean', 'D_73__last', 'B_21__last', 'B_28__mean', 'S_16__mean__log', 'R_3__last', 'D_41__std', 'D_60__last', 'D_75__first', 'R_24__std', 'D_44__last__log', 'D_131__last__log', 'R_2__max', 'D_43__mean__log', 'D_69__last', 'R_27__last', 'D_77__first', 'B_2__last', 'D_47__last', 'D_135__last__log', 'P_3__min', 'B_16__range', 'B_11__max', 'D_75__std', 'B_9__range', 'S_11__min', 'D_134__mean2std', 'B_4__last__log', 'R_9__last', 'D_55__range', 'R_1__range', 'B_2__last__log', 'R_9__mean2std', 'D_53__mean2std', 'D_120=1.0', 'R_20__max', 'D_44__last', 'B_11__mean__log', 'B_37__min', 'B_5__last', 'B_23__last', 'D_89__mean__log', 'S_17__min', 'P_4__range', 'D_48__mean', 'R_5__range', 'D_55__min', 'R_12__last', 'D_87__last', 'D_42__mean2std', 'S_5__last', 'D_39__mean__log', 'R_21__mean__log', 'S_23__mean', 'B_2__last2max', 'D_131__range', 'B_23__mean', 'D_112__mean', 'D_86__last', 'B_16__last__log', 'R_11__mean__log', 'S_20__mean__log', 'R_3__std', 'B_19__first', 'P_3__last2max', 'D_84__last', 'B_8__mean__log', 'B_23__first', 'B_26__last__log', 'R_6__last__log', 'B_40__min', 'D_79__range', 'B_7__last', 'D_39__mean2std', 'S_5__mean__log', 'D_108__last', 'D_45__last__log', 'D_62__mean', 'B_12__mean__log', 'B_8__min', 'R_15__last', 'B_27__last__log', 'R_7__std', 'D_68=1.0', 'B_4__range', 'D_103__last', 'D_81__mean__log', 'R_8__mean', 'S_3__range', 'D_84__mean__log', 'B_40__last__log', 'D_128__min', 'D_82__last', 'B_1__range', 'D_124__last', 'R_1__last__log', 'D_42__range', 'D_81__range', 'R_12__std', 'D_127__std', 'D_130__max', 'S_23__last', 'D_94__last', 'D_116__last', 'B_20__range', 'B_22__last__log', 'D_58__range', 'D_128__max', 'D_44__first', 'B_26__last', 'S_25__mean', 'R_26__last', 'B_24__max', 'D_128__last__log', 'D_78__last__log', 'D_54__std', 'R_18__max', 'D_52__first', 'D_43__max', 'D_53__mean', 'D_45__max', 'S_3__first', 'R_19__min', 'D_44__std', 'D_127__last__log', 'D_70__mean2std', 'B_38=6.0', 'B_5__mean', 'D_58__last__log', 'D_112__last2max', 'P_3__mean', 'B_15__mean', 'B_38__last', 'R_17__last', 'B_1__last2max', 'D_89__mean', 'B_25__last2max', 'D_51__last__log', 'D_53__last', 'D_59__min', 'S_13__last2max', 'B_11__last__log', 'B_8__last', 'R_3__mean__log', 'P_2__range', 'D_39__max', 'S_25__min', 'D_44__max', 'D_46__last', 'D_74__std', 'D_56__last', 'D_107__mean__log', 'D_58__last', 'D_133__last__log', 'B_1__last', 'D_55__std', 'D_84__mean', 'D_77__max', 'D_132__last', 'S_7__std', 'D_81__std', 'D_135__last', 'D_130__last', 'B_28__last', 'P_3__last', 'B_36__last', 'S_18__last', 'D_92__mean', 'D_46__min', 'D_117=999.0', 'B_9__last__log', 'B_20__first', 'B_6__min', 'D_79__max', 'D_113__mean', 'B_21__max', 'S_6__last', 'S_8__mean', 'D_79__std', 'R_5__std', 'R_26__mean2std', 'R_6__range', 'D_116__nunique', 'R_21__max', 'D_68__last', 'B_3__last__log', 'D_128__mean', 'D_39__std', 'D_122__last', 'R_3__min', 'R_1__mean', 'D_68=6.0', 'R_10__std', 'B_11__range', 'S_15__first', 'B_18__last2max', 'D_39__mean', 'R_11__range', 'B_19__max', 'D_128__mean__log', 'R_15__range', 'B_7__min', 'B_20__min', 'B_28__max', 'D_48__last2max', 'B_11__last2max', 'D_74__mean', 'D_114__last', 'D_72__std', 'S_7__max', 'D_65__mean__log', 'R_16__max', 'D_52__last', 'D_143__last', 'B_4__last', 'B_33__min', 'D_42__min', 'R_5__last', 'R_22__max', 'R_1__std', 'D_91__mean__log', 'R_11__max', 'D_109__last__log', 'P_2__first', 'B_21__last__log', 'D_51__mean', 'D_127__first', 'B_3__range', 'S_22__max', 'D_62__min', 'D_45__last', 'B_14__last', 'R_22__range', 'D_65__range', 'D_122__max', 'S_17__last', 'D_39__last', 'D_41__last', 'D_48__last', 'D_65__last', 'R_10__last', 'B_18__mean__log', 'B_6__mean', 'R_1__first', 'B_10__last2max', 'R_10__last__log', 'R_2__mean', 'D_54__range', 'D_77__mean__log', 'S_23__max', 'R_27__min', 'P_3__first', 'S_6__std', 'S_22__last', 'B_5__last__log', 'S_27__mean2std', 'D_50__last', 'D_51__max', 'S_24__max', 'D_49__last', 'B_18__min', 'R_27__max', 'D_41__mean__log', 'D_54__min', 'D_113__last__log', 'D_43__range', 'D_125__last', 'D_55__max', 'D_138__mean2std', 'B_7__std', 'B_22__max', 'D_89__max', 'D_45__mean', 'R_12__min', 'S_25__range', 'S_12__max', 'B_4__mean__log', 'D_132__min', 'D_64=O', 'B_32__last', 'D_81__last', 'B_33__mean__log', 'D_106__last', 'S_6__range', 'D_41__range', 'B_30=0.0', 'D_114=1.0', 'D_56__min', 'R_2__last2min', 'S_8__last', 'D_133__last', 'D_41__last__log', 'S_11__last', 'D_65__mean', 'D_74__first', 'B_4__std', 'D_51__std', 'B_22__std', 'D_42__last', 'D_53__last__log', 'S_23__std', 'D_54__last', 'S_9__max', 'D_44__range', 'D_54__last2max', 'P_2__last', 'D_58__std', 'D_115__last', 'D_45__first', 'D_59__max', 'D_118__last__log', 'D_58__mean__log', 'D_130__mean__log', 'S_7__mean', 'R_1__mean__log', 'R_3__max', 'B_3__std', 'D_107__last', 'D_75__min', 'R_17__range', 'D_91__last', 'S_27__last2max', 'P_2__std', 'R_8__max', 'D_84__std', 'D_112__mean__log', 'S_3__mean', 'R_21__mean', 'D_114__nunique', 'B_33__mean', 'D_44__mean', 'D_122__mean__log', 'D_130__first', 'R_4__last', 'R_11__mean', 'B_9__last', 'D_46__mean', 'S_9__last', 'D_64__last=U', 'B_9__mean__log', 'D_70__max', 'B_38=1.0', 'D_55__first', 'B_12__last__log', 'B_4__max', 'D_79__mean__log', 'B_15__last', 'B_10__first', 'B_6__last', 'B_24__last__log', 'D_44__min', 'B_37__mean', 'S_3__max', 'D_62__max', 'R_21__range', 'D_42__mean', 'D_79__last', 'B_4__min', 'D_74__last2max', 'B_1__mean', 'D_78__max', 'B_17__mean__log', 'D_84__last__log', 'R_16__mean', 'B_14__min', 'D_42__first', 'B_37__max', 'B_18__first', 'D_41__mean2std', 'D_55__mean', 'D_48__first', 'D_121__max', 'P_4__min', 'S_24__last', 'D_41__mean', 'R_8__last', 'B_2__std', 'R_1__last2min', 'D_55__last__log', 'D_77__last', 'S_15__mean', 'D_59__mean', 'B_28__last2max', 'B_7__last2max', 'B_22__last', 'D_74__max', 'B_16__mean__log', 'R_7__range', 'D_69__min', 'B_9__max', 'R_3__first', 'B_2__first', 'B_11__min', 'P_4__mean', 'B_3__min', 'D_53__mean__log', 'D_84__max', 'D_119__mean__log', 'D_106__last__log', 'R_3__range', 'B_23__last__log', 'B_40__last2max', 'D_118__first', 'D_43__mean', 'S_16__last', 'D_129__max', 'R_6__std', 'B_22__mean', 'D_42__max', 'B_10__last', 'S_13__min', 'R_5__last__log', 'B_11__mean', 'B_4__last2max', 'B_20__last', 'D_39__last__log', 'B_37__last', 'P_2__last2max', 'R_3__mean', 'B_20__last__log', 'B_16__first', 'P_2__mean', 'D_119__last__log', 'B_32__mean__log', 'D_116=999.0', 'R_16__range', 'D_44__last2max', 'B_22__min', 'D_75__range', 'D_43__std', 'D_75__last__log', 'R_20__last', 'D_55__mean__log', 'B_17__last2max', 'D_140__max', 'D_45__mean__log', 'B_11__last', 'B_38=7.0', 'D_131__first', 'S_13__last', 'R_2__last', 'D_134__min', 'R_7__max', 'R_24__mean', 'B_19__mean__log', 'B_2__range', 'D_41__max', 'D_120=0.0', 'S_25__std', 'D_137__last', 'S_25__last', 'B_2__mean', 'D_102__max', 'D_121__min', 'B_33__last__log', 'D_121__last', 'B_24__last', 'D_114=0.0', 'B_25__last', 'D_139__mean', 'R_20__mean__log', 'S_3__last', 'B_26__mean__log', 'D_115__last__log', 'R_16__mean__log', 'D_51__mean__log', 'D_75__last2max', 'D_112__last', 'D_145__last', 'B_3__last', 'D_56__mean2std', 'R_11__std', 'D_62__first', 'R_2__last__log', 'R_11__last', 'D_131__min', 'S_25__last2max', 'R_13__last', 'S_15__std', 'P_4__last__log', 'D_130__last__log']
feats715 = ['D_48__max', 'D_129__mean', 'D_129__last', 'D_107__last__log', 'D_49__last__log', 'B_1__max', 'D_112__range', 'D_43__min', 'D_52__max', 'D_78__last', 'D_39__range', 'S_15__range', 'B_2__max', 'D_113__max', 'B_18__last', 'B_12__last', 'R_15__mean__log', 'B_23__last2max', 'B_17__mean', 'S_24__mean', 'D_78__std', 'S_20__max', 'R_8__range', 'P_2__max', 'D_76__last', 'D_49__mean2std', 'B_9__min', 'R_6__mean__log', 'R_14__last__log', 'B_19__std', 'D_70__range', 'R_4__mean', 'D_131__mean', 'B_30=2.0', 'B_3__max', 'D_92__mean__log', 'D_133__max', 'B_18__last__log', 'P_4__first', 'D_92__max', 'D_74__last', 'B_28__min', 'D_121__first', 'D_131__max', 'D_79__mean', 'R_24__max', 'B_33__first', 'S_3__min', 'D_74__range', 'D_44__mean__log', 'S_11__mean', 'B_4__mean', 'S_15__max', 'S_8__last2max', 'D_119__min', 'D_43__last__log', 'D_115__min', 'R_2__mean__log', 'D_71__last__log', 'S_20__std', 'B_37__range', 'R_5__mean', 'D_130__min', 'D_71__last', 'R_13__mean2std', 'P_2__min', 'B_40__last', 'B_2__min', 'D_79__last__log', 'D_121__mean', 'D_73__last', 'B_21__last', 'B_28__mean', 'S_16__mean__log', 'R_3__last', 'D_41__std', 'D_60__last', 'D_75__first', 'R_24__std', 'D_44__last__log', 'D_70__last', 'S_6__min', 'D_53__range', 'D_131__last__log', 'R_4__last__log', 'R_2__max', 'D_43__mean__log', 'D_69__last', 'R_4__mean__log', 'R_27__last', 'D_77__first', 'B_2__last', 'D_47__last', 'D_135__last__log', 'P_3__min', 'B_16__range', 'B_11__max', 'D_75__std', 'B_9__range', 'S_11__min', 'D_134__mean2std', 'B_4__last__log', 'R_9__last', 'D_55__range', 'R_1__range', 'B_2__last__log', 'D_48__std', 'R_9__mean2std', 'S_6__max', 'D_53__mean2std', 'D_120=1.0', 'R_20__max', 'D_64=U', 'D_44__last', 'B_11__mean__log', 'B_37__min', 'B_5__last', 'B_23__last', 'D_89__mean__log', 'S_17__min', 'P_4__range', 'D_48__mean', 'R_5__range', 'D_55__min', 'R_12__last', 'D_87__last', 'D_42__mean2std', 'S_5__last', 'B_8__last__log', 'D_39__mean__log', 'R_21__mean__log', 'S_23__mean', 'B_2__last2max', 'D_131__range', 'B_23__mean', 'D_112__mean', 'D_86__last', 'B_16__last__log', 'R_11__mean__log', 'S_20__mean__log', 'R_3__std', 'B_19__min', 'B_19__first', 'P_3__last2max', 'D_137__last__log', 'S_11__std', 'D_84__last', 'B_8__mean__log', 'B_23__first', 'B_26__last__log', 'R_6__last__log', 'B_40__min', 'D_79__range', 'S_8__min', 'B_20__mean2std', 'B_7__last', 'D_39__mean2std', 'S_5__mean__log', 'D_108__last', 'D_61__last', 'S_3__std', 'D_45__last__log', 'B_30=1.0', 'D_62__mean', 'B_12__mean__log', 'R_10__mean__log', 'B_8__min', 'R_15__last', 'R_8__std', 'B_27__last__log', 'R_7__std', 'D_68=1.0', 'D_61__first', 'D_106__mean2std', 'B_4__range', 'D_103__last', 'D_81__mean__log', 'R_8__mean', 'S_3__range', 'D_84__mean__log', 'R_7__mean__log', 'B_40__last__log', 'D_128__min', 'D_78__first', 'D_82__last', 'D_58__last2max', 'B_1__range', 'D_124__last', 'R_1__last__log', 'D_42__range', 'D_81__range', 'R_12__std', 'D_127__std', 'D_130__max', 'S_23__last', 'D_94__last', 'R_15__mean', 'D_116__last', 'B_4__first', 'B_20__range', 'D_48__min', 'B_22__last__log', 'D_58__range', 'B_3__last2max', 'D_128__max', 'D_44__first', 'B_26__last', 'S_25__mean', 'R_26__last', 'B_24__max', 'B_19__mean', 'D_128__last__log', 'D_120__last', 'D_78__last__log', 'D_111__last', 'D_54__std', 'R_18__max', 'D_52__first', 'D_43__max', 'D_53__mean', 'D_45__max', 'B_20__last2max', 'D_80__last', 'S_3__first', 'R_19__min', 'D_44__std', 'D_74__last__log', 'D_127__last__log', 'D_77__range', 'D_70__mean2std', 'B_38=6.0', 'B_5__mean', 'D_58__last__log', 'D_112__last2max', 'P_3__mean', 'B_15__mean', 'B_38__last', 'R_17__last', 'B_23__std', 'B_1__last2max', 'D_89__mean', 'B_25__last2max', 'D_51__last__log', 'D_65__last__log', 'D_53__last', 'D_59__min', 'S_13__last2max', 'B_11__last__log', 'D_70__std', 'B_8__last', 'R_3__mean__log', 'P_2__range', 'D_71__mean__log', 'D_39__max', 'S_25__min', 'D_44__max', 'D_46__last', 'D_74__std', 'R_1__max', 'D_56__last', 'D_107__mean__log', 'D_58__last', 'D_133__last__log', 'B_1__last', 'D_55__std', 'P_4__max', 'D_60__last__log', 'D_84__mean', 'D_92__last', 'D_77__max', 'B_10__max', 'D_132__last', 'S_7__std', 'D_81__std', 'D_135__last', 'D_130__last', 'B_28__last', 'P_3__last', 'D_93__last', 'B_36__last', 'S_18__last', 'D_92__mean', 'D_46__min', 'D_117=999.0', 'B_9__last__log', 'R_20__std', 'B_20__first', 'B_6__min', 'D_79__max', 'D_113__mean', 'B_21__max', 'D_41__min', 'S_6__last', 'S_8__mean', 'D_79__std', 'R_5__std', 'R_26__mean2std', 'D_61__min', 'S_7__first', 'R_6__range', 'D_112__min', 'D_116__nunique', 'R_6__last', 'R_7__last', 'R_21__max', 'D_68__last', 'B_3__last__log', 'D_128__mean', 'D_39__std', 'D_122__last', 'R_3__min', 'R_1__mean', 'D_68=6.0', 'R_10__std', 'B_11__range', 'B_14__mean', 'S_15__first', 'B_18__last2max', 'D_39__mean', 'R_11__range', 'D_58__min', 'B_19__max', 'D_128__mean__log', 'R_15__range', 'B_7__min', 'B_20__min', 'D_133__mean__log', 'B_28__max', 'D_48__last2max', 'B_11__last2max', 'B_19__last', 'D_74__mean', 'D_114__last', 'D_72__std', 'S_7__max', 'D_65__mean__log', 'D_79__first', 'R_16__max', 'B_10__mean', 'D_52__last', 'S_15__last', 'B_19__last__log', 'B_8__max', 'D_143__last', 'D_130__mean', 'D_74__mean__log', 'B_4__last', 'D_51__first', 'B_33__min', 'D_42__min', 'R_5__last', 'R_22__max', 'B_21__mean__log', 'R_1__std', 'D_78__mean__log', 'D_91__mean__log', 'S_25__first', 'R_11__max', 'D_109__last__log', 'P_2__first', 'B_21__last__log', 'D_51__mean', 'D_127__first', 'B_3__range', 'S_27__last', 'S_22__max', 'B_38__nunique', 'D_62__min', 'D_45__last', 'B_14__last', 'R_22__range', 'D_65__range', 'D_144__last', 'S_17__last', 'D_122__max', 'D_39__last', 'D_41__last', 'B_17__last', 'D_48__last', 'D_89__std', 'D_65__last', 'R_10__last', 'B_18__mean__log', 'B_6__mean', 'R_1__first', 'R_1__last', 'B_10__last2max', 'R_10__last__log', 'R_2__mean', 'D_54__range', 'D_77__mean__log', 'B_16__last2max', 'S_23__max', 'R_27__min', 'B_2__mean__log', 'P_3__first', 'D_112__last__log', 'S_6__std', 'D_72__last', 'S_22__last', 'B_5__last__log', 'S_27__mean2std', 'D_50__last', 'D_51__max', 'S_24__max', 'D_49__last', 'B_18__min', 'R_27__max', 'D_41__mean__log', 'D_54__min', 'D_113__last__log', 'D_43__range', 'D_125__last', 'D_55__max', 'D_52__mean', 'D_138__mean2std', 'B_7__std', 'B_22__max', 'D_89__max', 'D_45__mean', 'R_12__min', 'R_13__range', 'R_14__mean__log', 'S_25__range', 'B_38=2.0', 'S_12__max', 'B_4__mean__log', 'D_132__min', 'D_64=O', 'B_32__last', 'D_81__last', 'B_33__mean__log', 'D_106__last', 'S_6__range', 'S_7__last', 'D_41__range', 'B_30=0.0', 'D_114=1.0', 'D_56__min', 'R_2__last2min', 'S_8__last', 'D_133__last', 'B_9__last2max', 'D_78__mean', 'B_6__last2max', 'R_4__max', 'D_41__last__log', 'S_11__last', 'R_21__std', 'D_72__max', 'B_9__mean', 'B_32__range', 'D_65__mean', 'D_74__first', 'B_4__std', 'B_23__max', 'D_51__std', 'B_22__std', 'B_24__mean__log', 'D_42__last', 'D_53__last__log', 'S_23__std', 'D_54__last', 'D_59__last', 'R_16__std', 'S_9__max', 'D_84__range', 'D_44__range', 'D_54__last2max', 'R_8__mean__log', 'P_2__last', 'D_58__std', 'D_115__last', 'D_45__first', 'D_59__max', 'D_118__last__log', 'D_58__mean__log', 'D_130__mean__log', 'S_7__mean', 'R_1__mean__log', 'R_3__max', 'B_3__std', 'D_107__last', 'D_72__mean__log', 'D_75__min', 'R_17__range', 'D_91__last', 'B_32__last__log', 'S_27__last2max', 'P_2__std', 'R_8__max', 'D_84__std', 'B_17__min', 'D_112__mean__log', 'S_3__mean', 'R_21__mean', 'D_114__nunique', 'B_33__mean', 'D_44__mean', 'D_122__mean__log', 'D_130__first', 'R_4__last', 'R_11__mean', 'B_9__last', 'D_46__mean', 'S_9__last', 'D_77__min', 'D_64__last=U', 'D_75__last', 'B_9__mean__log', 'D_70__max', 'B_38=1.0', 'D_55__first', 'B_12__last__log', 'B_4__max', 'D_79__mean__log', 'B_15__last', 'B_10__first', 'B_6__last', 'D_47__min', 'B_24__last__log', 'D_44__min', 'B_37__mean', 'B_14__last2max', 'D_109__last', 'S_3__max', 'D_62__max', 'R_10__range', 'R_21__range', 'D_42__mean', 'D_79__last', 'R_10__mean', 'B_37__std', 'B_5__mean__log', 'B_4__min', 'D_74__last2max', 'B_1__mean', 'D_78__max', 'B_17__mean__log', 'S_23__range', 'D_84__last__log', 'R_16__mean', 'B_14__min', 'D_42__first', 'B_37__max', 'B_18__first', 'D_41__mean2std', 'D_55__mean', 'D_48__first', 'D_121__max', 'P_4__min', 'S_24__last', 'D_115__mean', 'D_102__last', 'D_41__mean', 'R_8__last', 'B_2__std', 'R_1__last2min', 'D_55__last__log', 'D_77__last', 'S_15__mean', 'B_6__first', 'D_59__mean', 'R_15__std', 'B_28__last2max', 'B_7__last2max', 'B_22__last', 'B_29__last__log', 'D_47__max', 'B_25__min', 'D_74__max', 'D_64__last=O', 'R_7__mean', 'B_16__mean__log', 'R_7__range', 'D_69__min', 'R_25__range', 'B_9__max', 'R_3__first', 'D_47__first', 'B_2__first', 'B_11__min', 'P_4__mean', 'D_65__max', 'B_3__mean__log', 'D_54__mean', 'B_3__min', 'D_53__mean__log', 'B_23__min', 'D_84__max', 'B_16__min', 'D_119__mean__log', 'D_106__last__log', 'R_3__range', 'D_131__last', 'B_16__mean', 'B_23__last__log', 'B_40__last2max', 'D_118__first', 'D_43__mean', 'R_8__last__log', 'S_20__last', 'D_132__mean2std', 'D_122__mean', 'S_16__last', 'D_129__max', 'R_6__std', 'D_75__mean', 'B_22__mean', 'D_42__max', 'B_10__last', 'S_13__min', 'R_5__last__log', 'B_11__mean', 'R_2__first', 'B_4__last2max', 'B_20__last', 'D_39__last__log', 'B_7__first', 'B_37__last', 'P_2__last2max', 'R_3__mean', 'B_20__last__log', 'B_16__first', 'R_13__std', 'P_2__mean', 'D_55__last', 'D_119__last__log', 'B_32__mean__log', 'D_122__min', 'R_2__range', 'D_116=999.0', 'R_16__range', 'D_44__last2max', 'B_22__min', 'D_75__range', 'D_43__std', 'D_75__last__log', 'D_96__last', 'B_17__last__log', 'R_20__last', 'D_55__mean__log', 'B_17__last2max', 'D_140__max', 'D_45__mean__log', 'D_138__last', 'B_30__nunique', 'D_43__last', 'D_47__mean', 'B_11__last', 'B_38=7.0', 'D_131__first', 'S_13__last', 'R_2__last', 'D_134__min', 'R_6__max', 'R_7__max', 'R_24__mean', 'B_19__mean__log', 'B_2__range', 'B_1__first', 'D_41__max', 'D_70__mean', 'D_120=0.0', 'S_25__std', 'D_77__mean', 'D_137__last', 'D_74__min', 'S_25__last', 'B_33__last', 'B_2__mean', 'D_102__max', 'D_121__min', 'B_33__last__log', 'D_121__last', 'B_24__last', 'D_125__last__log', 'D_114=0.0', 'B_25__last', 'B_16__max', 'D_139__mean', 'R_20__mean__log', 'D_75__mean__log', 'R_10__first', 'S_3__last', 'B_26__mean__log', 'D_118__min', 'R_28__last__log', 'D_115__last__log', 'R_2__std', 'R_16__mean__log', 'P_4__std', 'D_65__std', 'D_51__mean__log', 'D_75__last2max', 'D_112__last', 'D_145__last', 'S_20__mean', 'D_131__mean__log', 'B_3__last', 'D_56__mean2std', 'R_11__std', 'D_62__first', 'R_2__last__log', 'R_11__last', 'D_131__min', 'S_25__last2max', 'R_13__last', 'S_15__std', 'P_4__last__log', 'D_123__last__log', 'D_58__max', 'D_130__last__log', 'B_25__mean', 'B_1__min']
feats485 = ['B_10__last', 'B_10__mean', 'B_11__last', 'B_11__last2max', 'B_11__last__log', 'B_11__mean', 'B_11__mean__log', 'B_11__min', 'B_12__last', 'B_12__last__log', 'B_13__last', 'B_13__last__log', 'B_14__last', 'B_14__last2max', 'B_14__min', 'B_15__last', 'B_15__mean', 'B_16__last', 'B_16__max', 'B_16__min', 'B_17__last', 'B_17__max', 'B_17__mean', 'B_17__min', 'B_18__last', 'B_18__last__log', 'B_18__mean', 'B_18__mean__log', 'B_18__min', 'B_19__last', 'B_19__last__log', 'B_19__min', 'B_1__last', 'B_1__last2max', 'B_1__max', 'B_1__mean', 'B_1__min', 'B_20__last', 'B_20__last__log', 'B_20__max', 'B_20__mean', 'B_20__mean__log', 'B_20__min', 'B_21__last', 'B_21__last__log', 'B_21__max', 'B_21__mean__log', 'B_22__last', 'B_22__last__log', 'B_22__max', 'B_22__mean', 'B_22__mean__log', 'B_22__min', 'B_23__last', 'B_23__last__log', 'B_23__mean', 'B_23__min', 'B_24__last', 'B_24__last__log', 'B_24__mean', 'B_24__mean__log', 'B_25__last', 'B_25__last2max', 'B_25__min', 'B_26__last', 'B_26__last__log', 'B_26__mean__log', 'B_27__last', 'B_27__last__log', 'B_28__last', 'B_2__last', 'B_2__last2max', 'B_2__last__log', 'B_2__mean', 'B_2__mean__log', 'B_2__min', 'B_30=0.0', 'B_32__last', 'B_32__last__log', 'B_32__max', 'B_33__last', 'B_33__last2max', 'B_33__last__log', 'B_33__max', 'B_33__mean', 'B_33__mean__log', 'B_33__min', 'B_36__last', 'B_36__last__log', 'B_37__last', 'B_37__last2max', 'B_37__max', 'B_37__mean', 'B_3__last', 'B_3__last__log', 'B_3__max', 'B_3__mean', 'B_3__mean__log', 'B_3__min', 'B_3__std', 'B_40__last', 'B_40__last__log', 'B_40__mean__log', 'B_40__min', 'B_41__last', 'B_41__last__log', 'B_4__last', 'B_4__last2max', 'B_4__last__log', 'B_4__max', 'B_4__mean__log', 'B_5__last', 'B_5__last__log', 'B_5__mean', 'B_6__last', 'B_6__min', 'B_7__last', 'B_7__max', 'B_7__mean', 'B_7__min', 'B_8__last', 'B_8__last__log', 'B_8__max', 'B_8__mean', 'B_8__mean__log', 'B_8__min', 'B_9__last', 'B_9__last2max', 'B_9__last__log', 'B_9__max', 'B_9__mean', 'B_9__mean__log', 'B_9__min', 'D_102__last', 'D_102__last__log', 'D_102__max', 'D_103__last', 'D_104__last', 'D_105__last', 'D_105__max', 'D_107__last', 'D_107__last__log', 'D_107__max', 'D_107__mean__log', 'D_109__last', 'D_109__last__log', 'D_109__mean', 'D_112__last', 'D_112__last__log', 'D_113__last', 'D_113__last__log', 'D_113__max', 'D_114=0.0', 'D_115__last', 'D_115__last__log', 'D_117__last', 'D_118__last', 'D_118__last__log', 'D_118__mean__log', 'D_119__last', 'D_119__last__log', 'D_121__last', 'D_122__last', 'D_122__max', 'D_122__min', 'D_123__last', 'D_123__last__log', 'D_123__mean', 'D_124__last', 'D_125__last', 'D_125__last__log', 'D_127__last', 'D_128__last', 'D_128__max', 'D_128__min', 'D_129__last', 'D_129__max', 'D_129__mean', 'D_130__last', 'D_130__max', 'D_131__last', 'D_131__last__log', 'D_131__max', 'D_131__min', 'D_133__last', 'D_133__last__log', 'D_133__max', 'D_133__min', 'D_139__last', 'D_139__mean', 'D_140__last', 'D_140__last__log', 'D_140__max', 'D_141__last', 'D_143__last', 'D_144__last', 'D_145__last', 'D_39__last', 'D_39__last__log', 'D_39__max', 'D_39__mean', 'D_39__range', 'D_39__std', 'D_41__last', 'D_41__last__log', 'D_41__max', 'D_41__mean', 'D_41__min', 'D_42__first', 'D_42__last', 'D_42__max', 'D_42__mean', 'D_42__min', 'D_43__last', 'D_43__last__log', 'D_43__max', 'D_43__mean', 'D_43__mean__log', 'D_43__min', 'D_44__last', 'D_44__last2max', 'D_44__last__log', 'D_44__max', 'D_44__mean', 'D_44__mean__log', 'D_44__min', 'D_44__range', 'D_44__std', 'D_45__first', 'D_45__last', 'D_45__last__log', 'D_45__max', 'D_45__mean', 'D_45__mean__log', 'D_45__min', 'D_46__last', 'D_46__mean', 'D_46__min', 'D_47__last', 'D_48__first', 'D_48__last', 'D_48__max', 'D_48__mean', 'D_48__min', 'D_49__last', 'D_49__last__log', 'D_50__last', 'D_51__last', 'D_51__last__log', 'D_51__mean', 'D_52__last', 'D_52__max', 'D_52__mean', 'D_52__min', 'D_53__last', 'D_53__last__log', 'D_53__max', 'D_54__last', 'D_55__last', 'D_55__last__log', 'D_55__max', 'D_55__min', 'D_56__last', 'D_56__min', 'D_58__last', 'D_58__min', 'D_59__last', 'D_59__max', 'D_60__last', 'D_60__last__log', 'D_60__max', 'D_61__last', 'D_61__max', 'D_61__mean', 'D_61__min', 'D_62__last', 'D_62__max', 'D_62__mean', 'D_62__min', 'D_64=U', 'D_65__last', 'D_65__last__log', 'D_65__max', 'D_65__mean', 'D_65__mean__log', 'D_69__last', 'D_69__min', 'D_70__last', 'D_70__max', 'D_70__min', 'D_71__last', 'D_72__last', 'D_72__max', 'D_72__min', 'D_74__last', 'D_74__max', 'D_74__mean', 'D_75__last', 'D_75__last__log', 'D_75__max', 'D_75__mean', 'D_75__mean__log', 'D_77__last', 'D_77__max', 'D_77__mean', 'D_77__min', 'D_78__last', 'D_78__max', 'D_78__mean', 'D_78__mean__log', 'D_79__last', 'D_79__max', 'D_80__last', 'D_81__last', 'D_81__max', 'D_81__mean', 'D_82__last', 'D_83__last', 'D_84__last', 'D_84__max', 'D_84__mean', 'D_86__last', 'D_89__last', 'D_89__max', 'D_89__mean', 'D_91__last', 'D_92__last', 'D_93__last', 'D_94__last', 'D_96__last', 'P_2__first', 'P_2__last', 'P_2__last2max', 'P_2__max', 'P_2__mean', 'P_2__min', 'P_3__last', 'P_3__max', 'P_3__mean', 'P_3__min', 'P_4__last', 'P_4__max', 'P_4__mean', 'P_4__min', 'R_10__last', 'R_10__max', 'R_10__mean', 'R_10__mean__log', 'R_10__std', 'R_11__last', 'R_11__max', 'R_11__mean', 'R_12__last', 'R_12__max', 'R_12__mean', 'R_13__last', 'R_13__max', 'R_13__mean', 'R_14__last', 'R_15__last', 'R_15__max', 'R_15__mean', 'R_16__last', 'R_16__max', 'R_16__mean', 'R_16__mean__log', 'R_17__last', 'R_17__max', 'R_18__last', 'R_18__max', 'R_19__last', 'R_19__min', 'R_1__last', 'R_1__last__log', 'R_1__max', 'R_1__mean', 'R_1__mean__log', 'R_1__min', 'R_1__range', 'R_1__std', 'R_20__last', 'R_20__max', 'R_20__mean', 'R_21__last', 'R_22__last', 'R_22__max', 'R_23__last', 'R_24__last', 'R_24__max', 'R_24__mean', 'R_25__last', 'R_25__max', 'R_27__last', 'R_27__max', 'R_27__mean', 'R_27__min', 'R_28__last', 'R_28__last__log', 'R_2__last', 'R_2__last__log', 'R_2__max', 'R_2__mean', 'R_2__mean__log', 'R_2__min', 'R_2__range', 'R_2__std', 'R_3__last', 'R_3__last__log', 'R_3__max', 'R_3__mean', 'R_3__mean__log', 'R_3__min', 'R_4__last', 'R_4__max', 'R_4__mean', 'R_4__mean__log', 'R_4__min', 'R_5__last', 'R_5__max', 'R_5__mean', 'R_5__mean__log', 'R_5__range', 'R_6__last', 'R_6__last__log', 'R_6__max', 'R_6__mean', 'R_6__mean__log', 'R_7__last', 'R_7__max', 'R_7__mean', 'R_7__range', 'R_8__last', 'R_8__max', 'R_8__mean', 'R_8__mean__log', 'S_11__last', 'S_11__mean', 'S_12__last', 'S_12__max', 'S_13__last', 'S_15__last', 'S_15__max', 'S_15__mean', 'S_16__last', 'S_17__last', 'S_17__min', 'S_18__last', 'S_19__last', 'S_20__last', 'S_20__max', 'S_22__last', 'S_22__max', 'S_22__mean', 'S_23__last', 'S_23__max', 'S_23__mean', 'S_24__last', 'S_25__last', 'S_25__last2max', 'S_25__mean', 'S_25__min', 'S_25__range', 'S_25__std', 'S_26__last', 'S_26__last__log', 'S_27__last', 'S_3__last', 'S_3__max', 'S_3__mean', 'S_3__min', 'S_5__last', 'S_5__last__log', 'S_5__mean__log', 'S_6__last', 'S_6__max', 'S_6__min', 'S_7__last', 'S_7__max', 'S_7__mean', 'S_7__min', 'S_8__last', 'S_8__mean', 'S_8__min', 'S_9__last', 'S_9__max']

garbage404 = ['D_48__max', 'R_3__last__log', 'D_113__last', 'B_39__last', 'D_128__first', 'B_42__last__log', 'B_18__last', 'R_23__last', 'R_15__mean__log', 'B_23__last2max', 'R_13__mean__log', 'R_17__mean__log', 'S_24__mean', 'S_20__max', 'R_8__range', 'D_115__max', 'D_113__min', 'D_127__range', 'R_4__mean', 'D_53__max', 'D_133__max', 'B_18__last__log', 'D_92__max', 'B_32__max', 'D_121__first', 'D_131__max', 'R_24__max', 'D_127__max', 'R_20__range', 'R_13__max', 'S_5__last__log', 'D_119__min', 'D_107__max', 'R_27__mean', 'S_6__last2max', 'B_37__range', 'D_91__max', 'B_40__last', 'P_3__std', 'D_52__min', 'B_38=5.0', 'D_120=999.0', 'D_70__last', 'S_6__min', 'D_53__range', 'R_7__last__log', 'R_4__last__log', 'D_104__last', 'R_4__mean__log', 'D_77__last__log', 'D_48__std', 'S_6__max', 'S_26__last', 'R_28__last', 'B_24__mean', 'R_14__last', 'D_64=U', 'B_29__last', 'D_102__last__log', 'D_139__last', 'S_22__first', 'D_81__max', 'B_8__last__log', 'D_62__last2max', 'D_83__last', 'B_7__mean', 'D_70__last__log', 'R_24__last', 'D_113__mean__log', 'B_19__min', 'S_11__std', 'D_137__last__log', 'B_20__mean__log', 'B_16__last', 'D_119__first', 'S_8__min', 'D_127__last', 'B_20__mean2std', 'D_89__last', 'D_61__last', 'S_3__std', 'D_113__first', 'B_30=1.0', 'R_10__mean__log', 'R_8__std', 'D_120__nunique', 'R_12__mean', 'B_10__min', 'D_61__first', 'D_106__mean2std', 'R_7__mean__log', 'D_78__first', 'B_20__std', 'R_24__range', 'B_22__first', 'D_58__last2max', 'R_2__min', 'D_61__mean', 'R_15__mean', 'R_13__mean', 'D_48__range', 'B_4__first', 'D_48__min', 'S_27__std', 'D_136__last', 'D_58__first', 'R_4__std', 'B_3__last2max', 'B_3__first', 'B_33__last2max', 'D_46__mean2std', 'D_109__mean', 'B_19__mean', 'B_37__first', 'D_120__last', 'B_36__last__log', 'D_111__last', 'B_20__last2max', 'D_80__last', 'B_9__std', 'D_74__last__log', 'D_77__range', 'B_23__std', 'B_22__mean__log', 'D_65__last__log', 'D_127__mean', 'B_28__first', 'D_81__last__log', 'D_127__mean__log', 'D_70__std', 'D_53__std', 'D_71__mean__log', 'S_12__last', 'R_1__max', 'S_23__mean2std', 'B_23__range', 'P_4__max', 'D_60__last__log', 'D_92__last', 'R_21__last', 'B_10__max', 'D_68__nunique', 'R_5__mean__log', 'D_140__last', 'B_7__range', 'B_27__last', 'D_142__last', 'D_119__mean', 'P_3__range', 'D_93__last', 'D_61__max', 'D_70__min', 'D_53__first', 'R_20__std', 'D_75__max', 'D_42__std', 'D_41__min', 'B_32__std', 'D_61__min', 'S_7__first', 'D_112__min', 'D_118__max', 'R_10__max', 'S_27__last__log', 'R_7__last', 'R_6__last', 'D_123__mean', 'B_7__max', 'B_14__mean', 'B_13__last__log', 'B_3__mean', 'D_51__min', 'D_58__min', 'D_133__mean__log', 'B_19__last', 'B_24__range', 'D_108__last__log', 'D_136__last__log', 'D_72__last__log', 'D_115__mean__log', 'R_15__max', 'D_79__first', 'P_3__max', 'B_10__mean', 'S_15__last', 'B_19__last__log', 'B_8__max', 'R_25__max', 'D_74__mean__log', 'D_130__mean', 'R_12__range', 'R_17__mean', 'D_51__first', 'D_136__mean2std', 'R_4__range', 'D_128__last', 'S_26__last__log', 'B_21__mean__log', 'D_78__mean__log', 'S_25__first', 'D_134__last', 'S_27__last', 'B_38__nunique', 'B_9__first', 'D_144__last', 'R_1__min', 'B_17__last', 'D_72__range', 'D_119__max', 'D_89__std', 'B_18__max', 'B_1__std', 'D_122__first', 'R_1__last', 'D_70__first', 'D_72__min', 'R_16__last', 'D_55__last2max', 'P_4__mean__log', 'B_17__max', 'B_16__last2max', 'B_2__mean__log', 'B_33__max', 'D_112__last__log', 'D_72__last', 'B_6__max', 'D_51__range', 'D_52__mean', 'R_6__mean', 'R_13__range', 'R_14__mean__log', 'B_38=2.0', 'S_7__last', 'R_20__mean', 'D_78__range', 'B_9__last2max', 'D_62__last', 'D_78__mean', 'R_4__max', 'B_6__last2max', 'D_72__max', 'B_9__mean', 'R_21__std', 'R_4__min', 'D_140__last__log', 'B_37__last2max', 'B_32__range', 'B_38=4.0', 'D_45__min', 'B_23__max', 'B_41__last__log', 'B_24__mean__log', 'R_19__last', 'D_59__last', 'R_16__std', 'B_17__mean2std', 'D_84__range', 'B_8__last2max', 'R_8__mean__log', 'B_8__mean', 'D_72__mean__log', 'B_32__last__log', 'S_7__range', 'B_17__min', 'B_20__mean', 'D_61__last2max', 'R_27__first', 'R_22__last', 'B_11__std', 'B_19__range', 'B_24__std', 'D_72__mean', 'R_24__mean__log', 'D_77__min', 'D_133__min', 'D_75__last', 'B_31__nunique', 'D_79__min', 'R_12__max', 'D_115__first', 'B_4__mean2std', 'B_8__first', 'D_47__min', 'B_14__last2max', 'D_109__last', 'D_81__mean', 'S_22__mean', 'D_118__last', 'R_10__range', 'R_10__mean', 'B_37__std', 'B_5__mean__log', 'D_112__std', 'B_40__first', 'S_23__range', 'D_114=999.0', 'B_20__max', 'D_60__max', 'D_51__last', 'B_16__std', 'D_115__mean', 'D_126__last', 'D_102__last', 'D_123__last', 'B_13__last', 'S_7__min', 'B_6__first', 'R_15__std', 'B_29__last__log', 'D_47__max', 'B_41__last', 'B_25__min', 'D_64__last=O', 'R_7__mean', 'D_105__max', 'R_25__std', 'D_141__last', 'R_25__range', 'B_40__max', 'D_47__first', 'B_23__mean__log', 'D_65__max', 'R_17__std', 'B_3__mean__log', 'B_23__min', 'D_126=999.0', 'D_54__mean', 'B_16__min', 'B_11__first', 'B_42__last', 'D_131__last', 'B_16__mean', 'P_4__last', 'D_62__std', 'S_20__last', 'R_8__last__log', 'B_40__mean__log', 'D_132__mean2std', 'D_138__last__log', 'D_122__mean', 'D_119__last', 'B_40__mean', 'D_75__mean', 'D_89__range', 'S_19__last', 'D_105__last', 'R_18__last', 'R_2__first', 'D_118__mean__log', 'B_22__range', 'B_7__first', 'D_43__first', 'D_52__mean2std', 'R_13__std', 'D_55__last', 'R_17__max', 'D_122__min', 'D_110__last', 'R_2__range', 'D_88__last', 'B_18__mean', 'D_59__first', 'D_96__last', 'D_118__mean', 'B_17__last__log', 'B_31__last', 'D_138__last', 'B_30__nunique', 'D_43__last', 'D_47__mean', 'D_58__mean', 'R_6__max', 'B_30__last', 'B_1__first', 'D_70__mean__log', 'D_135__mean2std', 'D_70__mean', 'D_77__mean', 'D_74__min', 'B_33__last', 'D_117__last', 'R_5__max', 'D_125__last__log', 'B_16__max', 'D_75__mean__log', 'D_53__min', 'R_10__first', 'D_118__min', 'D_62__range', 'R_28__last__log', 'R_2__std', 'P_4__std', 'D_65__std', 'R_25__last', 'S_20__mean', 'D_131__mean__log', 'B_19__last2max', 'D_137__mean2std', 'S_20__range', 'D_131__std', 'D_123__last__log', 'D_58__max', 'B_25__mean', 'B_1__min']
garbage38 = ['B_23__max', 'D_113__min', 'P_4__max', 'R_13__max', 'D_77__min', 'R_16__last', 'B_33__last', 'D_64=U', 'D_118__mean', 'D_127__range', 'D_118__mean__log', 'D_127__mean__log', 'B_23__min', 'D_79__min', 'D_115__max', 'R_17__mean', 'D_105__last', 'B_23__mean__log', 'B_29__last', 'B_20__std', 'B_7__range', 'D_48__min', 'B_32__last__log', 'R_15__mean', 'R_15__mean__log', 'R_13__range', 'D_133__mean__log', 'D_113__mean__log', 'B_13__last__log', 'D_108__last__log', 'B_8__last__log', 'D_74__min', 'D_118__min', 'B_20__last2max', 'D_53__range', 'R_7__mean', 'B_3__first', 'R_14__last']
garbage20 = ['D_113__last', 'B_20__first', 'D_121__max', 'S_15__first', 'D_72__std', 'D_70__mean2std', 'D_128__mean', 'D_68__nunique', 'D_109__mean', 'B_36__last__log', 'D_115__max', 'R_11__last', 'B_36__last', 'R_9__mean2std', 'D_88__last', 'D_136__last__log', 'D_94__last', 'D_82__last', 'D_113__first', 'S_6__range']

feats551 = ['B_10__last', 'B_10__mean', 'B_11__last', 'B_11__last2max', 'B_11__last__log', 'B_11__mean', 'B_11__mean__log', 'B_11__min', 'B_12__last', 'B_12__last__log', 'B_13__last', 'B_13__last__log', 'B_14__last', 'B_14__last2max', 'B_14__min', 'B_15__last', 'B_15__mean', 'B_16__last', 'B_16__max', 'B_16__min', 'B_17__last', 'B_17__max', 'B_17__mean', 'B_17__mean2std', 'B_17__min', 'B_18__last', 'B_18__last__log', 'B_18__mean', 'B_18__mean__log', 'B_18__min', 'B_19__last', 'B_19__last__log', 'B_19__min', 'B_1__last', 'B_1__last2max', 'B_1__max', 'B_1__mean', 'B_1__min', 'B_20__last', 'B_20__last__log', 'B_20__max', 'B_20__mean', 'B_20__mean__log', 'B_20__min', 'B_21__last', 'B_21__last__log', 'B_21__max', 'B_21__mean__log', 'B_22__last', 'B_22__last__log', 'B_22__max', 'B_22__mean', 'B_22__mean__log', 'B_22__min', 'B_23__last', 'B_23__last__log', 'B_23__mean', 'B_23__min', 'B_24__last', 'B_24__last__log', 'B_24__mean', 'B_24__mean__log', 'B_25__last', 'B_25__last2max', 'B_25__min', 'B_26__last', 'B_26__last__log', 'B_26__mean__log', 'B_27__last', 'B_27__last__log', 'B_28__last', 'B_29__last', 'B_29__last__log', 'B_2__last', 'B_2__last2max', 'B_2__last__log', 'B_2__mean', 'B_2__mean__log', 'B_2__min', 'B_30=0.0', 'B_30=1.0', 'B_30=2.0', 'B_30__last', 'B_30__nunique', 'B_31__last', 'B_31__nunique', 'B_32__last', 'B_32__last__log', 'B_32__max', 'B_33__last', 'B_33__last2max', 'B_33__last__log', 'B_33__max', 'B_33__mean', 'B_33__mean__log', 'B_33__min', 'B_36__last', 'B_36__last__log', 'B_37__last', 'B_37__last2max', 'B_37__max', 'B_37__mean', 'B_38=4.0', 'B_38=5.0', 'B_38=6.0', 'B_38=7.0', 'B_38__last', 'B_38__nunique', 'B_39__last', 'B_3__last', 'B_3__last__log', 'B_3__max', 'B_3__mean', 'B_3__mean__log', 'B_3__min', 'B_3__std', 'B_40__last', 'B_40__last__log', 'B_40__mean__log', 'B_40__min', 'B_41__last', 'B_41__last__log', 'B_42__last', 'B_42__last__log', 'B_4__last', 'B_4__last2max', 'B_4__last__log', 'B_4__max', 'B_4__mean2std', 'B_4__mean__log', 'B_5__last', 'B_5__last__log', 'B_5__mean', 'B_6__last', 'B_6__min', 'B_7__last', 'B_7__max', 'B_7__mean', 'B_7__min', 'B_8__last', 'B_8__last__log', 'B_8__max', 'B_8__mean', 'B_8__mean__log', 'B_8__min', 'B_9__last', 'B_9__last2max', 'B_9__last__log', 'B_9__max', 'B_9__mean', 'B_9__mean__log', 'B_9__min', 'D_102__last', 'D_102__last__log', 'D_102__max', 'D_103__last', 'D_104__last', 'D_105__last', 'D_105__max', 'D_106__last', 'D_106__last__log', 'D_107__last', 'D_107__last__log', 'D_107__max', 'D_107__mean__log', 'D_108__last', 'D_108__last__log', 'D_109__last', 'D_109__last__log', 'D_109__mean', 'D_110__last', 'D_111__last', 'D_112__last', 'D_112__last__log', 'D_113__last', 'D_113__last__log', 'D_113__max', 'D_114=0.0', 'D_114__last', 'D_114__nunique', 'D_115__last', 'D_115__last__log', 'D_116__last', 'D_117__last', 'D_118__last', 'D_118__last__log', 'D_118__mean__log', 'D_119__last', 'D_119__last__log', 'D_120=1.0', 'D_120__last', 'D_120__nunique', 'D_121__last', 'D_122__last', 'D_122__max', 'D_122__min', 'D_123__last', 'D_123__last__log', 'D_123__mean', 'D_124__last', 'D_125__last', 'D_125__last__log', 'D_126__last', 'D_127__last', 'D_128__last', 'D_128__max', 'D_128__min', 'D_129__last', 'D_129__max', 'D_129__mean', 'D_130__last', 'D_130__max', 'D_131__last', 'D_131__last__log', 'D_131__max', 'D_131__min', 'D_132__last', 'D_132__mean2std', 'D_132__min', 'D_133__last', 'D_133__last__log', 'D_133__max', 'D_133__min', 'D_134__last', 'D_134__min', 'D_135__last', 'D_135__last__log', 'D_135__mean2std', 'D_136__last', 'D_136__last__log', 'D_137__last', 'D_137__last__log', 'D_138__last', 'D_138__last__log', 'D_139__last', 'D_139__mean', 'D_140__last', 'D_140__last__log', 'D_140__max', 'D_141__last', 'D_142__last', 'D_143__last', 'D_144__last', 'D_145__last', 'D_39__last', 'D_39__last__log', 'D_39__max', 'D_39__mean', 'D_39__range', 'D_39__std', 'D_41__last', 'D_41__last__log', 'D_41__max', 'D_41__mean', 'D_41__min', 'D_42__first', 'D_42__last', 'D_42__max', 'D_42__mean', 'D_42__mean2std', 'D_42__min', 'D_43__last', 'D_43__last__log', 'D_43__max', 'D_43__mean', 'D_43__mean__log', 'D_43__min', 'D_44__last', 'D_44__last2max', 'D_44__last__log', 'D_44__max', 'D_44__mean', 'D_44__mean__log', 'D_44__min', 'D_44__range', 'D_44__std', 'D_45__first', 'D_45__last', 'D_45__last__log', 'D_45__max', 'D_45__mean', 'D_45__mean__log', 'D_45__min', 'D_46__last', 'D_46__mean', 'D_46__mean2std', 'D_46__min', 'D_47__last', 'D_48__first', 'D_48__last', 'D_48__max', 'D_48__mean', 'D_48__min', 'D_49__last', 'D_49__last__log', 'D_49__mean2std', 'D_50__last', 'D_51__last', 'D_51__last__log', 'D_51__mean', 'D_52__last', 'D_52__max', 'D_52__mean', 'D_52__mean2std', 'D_52__min', 'D_53__last', 'D_53__last__log', 'D_53__max', 'D_53__mean2std', 'D_53__min', 'D_54__last', 'D_55__last', 'D_55__last__log', 'D_55__max', 'D_55__min', 'D_56__last', 'D_56__min', 'D_58__last', 'D_58__min', 'D_59__last', 'D_59__max', 'D_60__last', 'D_60__last__log', 'D_60__max', 'D_61__last', 'D_61__max', 'D_61__mean', 'D_61__min', 'D_62__last', 'D_62__max', 'D_62__mean', 'D_62__min', 'D_64=U', 'D_65__last', 'D_65__last__log', 'D_65__max', 'D_65__mean', 'D_65__mean__log', 'D_68=1.0', 'D_68__last', 'D_69__last', 'D_69__min', 'D_70__last', 'D_70__max', 'D_70__mean2std', 'D_70__min', 'D_71__last', 'D_72__last', 'D_72__max', 'D_72__min', 'D_73__last', 'D_74__last', 'D_74__max', 'D_74__mean', 'D_75__last', 'D_75__last__log', 'D_75__max', 'D_75__mean', 'D_75__mean__log', 'D_76__last', 'D_77__last', 'D_77__max', 'D_77__mean', 'D_77__min', 'D_78__last', 'D_78__max', 'D_78__mean', 'D_78__mean__log', 'D_79__last', 'D_79__max', 'D_80__last', 'D_81__last', 'D_81__max', 'D_81__mean', 'D_82__last', 'D_83__last', 'D_84__last', 'D_84__max', 'D_84__mean', 'D_86__last', 'D_87__last', 'D_88__last', 'D_89__last', 'D_89__max', 'D_89__mean', 'D_91__last', 'D_92__last', 'D_93__last', 'D_94__last', 'D_96__last', 'P_2__first', 'P_2__last', 'P_2__last2max', 'P_2__max', 'P_2__mean', 'P_2__min', 'P_3__last', 'P_3__max', 'P_3__mean', 'P_3__min', 'P_4__last', 'P_4__max', 'P_4__mean', 'P_4__min', 'R_10__last', 'R_10__max', 'R_10__mean', 'R_10__mean__log', 'R_10__std', 'R_11__last', 'R_11__max', 'R_11__mean', 'R_12__last', 'R_12__max', 'R_12__mean', 'R_13__last', 'R_13__max', 'R_13__mean', 'R_13__mean2std', 'R_14__last', 'R_15__last', 'R_15__max', 'R_15__mean', 'R_16__last', 'R_16__max', 'R_16__mean', 'R_16__mean__log', 'R_17__last', 'R_17__max', 'R_18__last', 'R_18__max', 'R_19__last', 'R_19__min', 'R_1__last', 'R_1__last__log', 'R_1__max', 'R_1__mean', 'R_1__mean__log', 'R_1__min', 'R_1__range', 'R_1__std', 'R_20__last', 'R_20__max', 'R_20__mean', 'R_21__last', 'R_22__last', 'R_22__max', 'R_23__last', 'R_24__last', 'R_24__max', 'R_24__mean', 'R_25__last', 'R_25__max', 'R_26__last', 'R_26__mean2std', 'R_27__last', 'R_27__max', 'R_27__mean', 'R_27__min', 'R_28__last', 'R_28__last__log', 'R_2__last', 'R_2__last__log', 'R_2__max', 'R_2__mean', 'R_2__mean__log', 'R_2__min', 'R_2__range', 'R_2__std', 'R_3__last', 'R_3__last__log', 'R_3__max', 'R_3__mean', 'R_3__mean__log', 'R_3__min', 'R_4__last', 'R_4__max', 'R_4__mean', 'R_4__mean__log', 'R_4__min', 'R_5__last', 'R_5__max', 'R_5__mean', 'R_5__mean__log', 'R_5__range', 'R_6__last', 'R_6__last__log', 'R_6__max', 'R_6__mean', 'R_6__mean__log', 'R_7__last', 'R_7__max', 'R_7__mean', 'R_7__range', 'R_8__last', 'R_8__max', 'R_8__mean', 'R_8__mean__log', 'R_9__last', 'R_9__mean2std', 'S_11__last', 'S_11__mean', 'S_12__last', 'S_12__max', 'S_13__last', 'S_15__last', 'S_15__max', 'S_15__mean', 'S_16__last', 'S_17__last', 'S_17__min', 'S_18__last', 'S_19__last', 'S_20__last', 'S_20__max', 'S_22__last', 'S_22__max', 'S_22__mean', 'S_23__last', 'S_23__max', 'S_23__mean', 'S_24__last', 'S_25__last', 'S_25__last2max', 'S_25__mean', 'S_25__min', 'S_25__range', 'S_25__std', 'S_26__last', 'S_26__last__log', 'S_27__last', 'S_27__mean2std', 'S_3__last', 'S_3__max', 'S_3__mean', 'S_3__min', 'S_5__last', 'S_5__last__log', 'S_5__mean__log', 'S_6__last', 'S_6__max', 'S_6__min', 'S_7__last', 'S_7__max', 'S_7__mean', 'S_7__min', 'S_8__last', 'S_8__mean', 'S_8__min', 'S_9__last', 'S_9__max']

len(feats486), len(feats497)

(486, 497)

In [12]:

feats = feats551

len(feats)

551

In [ ]:
%%time

df = pd.read_parquet(train_file, columns=['customer_ID', 'target'] + feats, engine='pyarrow')
#df=pd.read_parquet(train_file, engine='pyarrow')


for params, model_file in zip(binary_params, model_files):
    num_boost_round = params['n_estimators']
    params_ = copy.deepcopy(params)
    del params_['n_estimators']


    #-----------------lgb train start-------------------------------------

    train_trees(df[x_cols].values, df['target'].values, 
                num_round=num_boost_round, params=params_, 
                verbosity=-1, 
                save_model = True, model_file = model_file)
    
del df
gc.collect()

In [ ]:
%%time
data = pd.read_parquet(test_file,columns=['customer_ID'] + feats,  engine='pyarrow')
# data = data[['customer_ID']+x_cols].copy(deep=True)
all_preds = inference_trees(data[feats].values, model_files)

test_preds = pd.DataFrame(data={'customer_ID': data['customer_ID'].values, 'prediction':all_preds.mean(axis=0)})
test_preds.to_csv('predictions/trees.csv', index=False)

del data, all_preds, test_preds
gc.collect()

## MLP

In [ ]:
import torch
from torch.utils.data import (Dataset, DataLoader)
import torch
from torch import nn
import numpy as np
 

class Train_Data(Dataset):
    
    def __init__(self, X, y): 
        
        features = torch.FloatTensor(X)
        targets = torch.FloatTensor(y)
        
        self.features = features
        self.targets = targets
        
        self.n_samples = X.shape[0]
        self.n_features = X.shape[1]
        
    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        

        x = self.features[idx]
        y = self.targets[idx]
        
        return x, y
    

def load_traindata(X, y, batch_size, n_workers=0, shuffle=False):
    data = Train_Data(X, y)
    
    loader = DataLoader(data, batch_size=batch_size, num_workers=n_workers, shuffle=shuffle)
    
    return loader


class Test_Data(Dataset):
    
    def __init__(self, X): 
        
        features = torch.FloatTensor(X)
        
        self.features = features
        
        self.n_samples = X.shape[0]
        self.n_features = X.shape[1]
        
    def __len__(self):
        return self.n_samples

    def __getitem__(self, idx):
        

        x = self.features[idx]
        
        return x
    

def load_testdata(X, batch_size, n_workers=0, shuffle=False):
    data = Test_Data(X)
    
    loader = DataLoader(data, batch_size=batch_size, num_workers=n_workers, shuffle=shuffle)
    
    return loader



class MLP(nn.Module):
    def __init__(
        self,
        input_size: int,
        output_size: int,
        hidden_sizes: list,
        dropouts: list,
    ):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_sizes = hidden_sizes
        self.dropouts = dropouts
        

        #---MLP--------------------------
                
        # input layer
        module_list = [nn.BatchNorm1d(input_size), 
                       nn.Dropout(dropouts[0]), 
                       nn.Linear(input_size, hidden_sizes[0]), 
                       nn.BatchNorm1d(hidden_sizes[0]), 
                       nn.SiLU(), 
                       nn.Dropout(dropouts[0])]
    
        # hidden layers
        for i in range(1, len(hidden_sizes)):
            module_list.extend([nn.Linear(hidden_sizes[i-1], hidden_sizes[i]), 
                                nn.BatchNorm1d(hidden_sizes[i]), 
                                nn.SiLU(), 
                                nn.Dropout(dropouts[i])]
                              )
        # output layer
        module_list.extend([nn.Linear(hidden_sizes[-1], output_size), 
                           nn.Sigmoid()])

        self.mlp = nn.Sequential(*module_list)
        
    def forward(self, x):
        out = self.mlp(x)

        return out


In [ ]:
def train_mlp(params, seed, train_loader, model_file):
    pl.seed_everything(seed)
    print(params)
    
    learning_rate = params['learning_rate']
    num_epochs = params['num_epochs']
    batch_size = params['batch_size']
    n_layers = params['n_layers']
    h_sizes = [params[f'hidden_size{i}'] for i in range(1,n_layers+1)]
    drop_list = [params[f'dropout{i}'] for i in range(1,n_layers+1)]



    model = MLP(input_size=len(feats), output_size=1, 
                   hidden_sizes=h_sizes, 
                   dropouts = drop_list)

    model = model.to(device)

    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)  
#         optimizer = torch.optim.RMSprop([
#                 {'params': model.mlp.parameters()},
#             ], lr=learning_rate)


#         scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
#                                                   max_lr=1e-2, epochs=num_epochs, steps_per_epoch=len(train_loader))
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)

    out_loss = BCELoss()


    #------train models--------------------------
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (features, targets) in enumerate(train_loader):
            optimizer.zero_grad()
            features = features.to(device)
            targets = targets.to(device)
            ### FORWARD AND BACK PROP
            out = model(features)
            out_cost = out_loss(out.squeeze(), targets) #squeeze the n_samples*1 2d array to 1d array of n_samples
            out_cost.backward()

            ### UPDATE MODEL PARAMETERS
            optimizer.step()
            scheduler.step()

    #-------save model to a file-----
    torch.save(model.state_dict(), model_file)
    
    
def inference_mlp(params, seed, test_loader, model_file):
    #-----load model file
    pl.seed_everything(seed)
    
    learning_rate = params['learning_rate']
    num_epochs = params['num_epochs']
    batch_size = params['batch_size']
    n_layers = params['n_layers']
    h_sizes = [params[f'hidden_size{i}'] for i in range(1,n_layers+1)]
    drop_list = [params[f'dropout{i}'] for i in range(1,n_layers+1)]
    
    model = MLP(input_size=len(feats), output_size=1, 
                   hidden_sizes=h_sizes, 
                   dropouts = drop_list)
    
    model.load_state_dict(torch.load(model_file, map_location = torch.device('cpu')))

    model = model.to(device)

    
    #-----eval models-------------------------------
    model.eval()

    y_preds = []
    with torch.no_grad():
        for features in test_loader:
            features = features.to(device)
            outputs = model(features)
            y_preds.extend(outputs.squeeze().cpu().numpy())


    y_preds = np.array(y_preds)
    return y_preds

In [ ]:
%%time

mlp_params =[
    {'batch_size': 4096, 'dropout1': 0.489, 'dropout2': 0.254, 'dropout3': 0.006, 'dropout4': 0.002, 'dropout5': 0.372, 'dropout6': 0.375, 'dropout7': 0.131, 'dropout8': 0.496, 'hidden_size1': 1024, 'hidden_size2': 512, 'hidden_size3': 48, 'hidden_size4': 512, 'hidden_size5': 48, 'hidden_size6': 896, 'hidden_size7': 256, 'hidden_size8': 256, 'learning_rate': 0.006, 'n_layers': 7, 'num_epochs': 20},
    {'batch_size': 1024, 'dropout1': 0.385, 'dropout2': 0.321, 'dropout3': 0.193, 'dropout4': 0.28, 'dropout5': 0.082, 'dropout6': 0.165, 'dropout7': 0.001, 'dropout8': 0.263, 'hidden_size1': 896, 'hidden_size2': 1024, 'hidden_size3': 1024, 'hidden_size4': 256, 'hidden_size5': 1024, 'hidden_size6': 96, 'hidden_size7': 448, 'hidden_size8': 896, 'learning_rate': 0.0002, 'n_layers': 5, 'num_epochs': 35}, 
]

print(len(mlp_params))

####==================1. load data===================================

df = pd.read_parquet(train_file, columns=['customer_ID', 'target'] + feats, engine='pyarrow')
#df=pd.read_parquet(train_file, engine='pyarrow')

X = df[feats]
y = df[['target']]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

scaler_ = StandardScaler()
scaler_.fit(X)

####==================2. train models ===================================
for i, params in enumerate(params_list):
    batch_size = params['batch_size']
    train_loader = load_traindata(scaler_.transform(X), y['target'].values, 
                         batch_size=batch_size, n_workers=0, shuffle=False)
    for seed in [1, 2001]:
        train_mlp(params, seed, train_loader, f'{i}_mlp_{seed}.pth')

del df, X, y, train_loader
gc.collect()
####==================3. predictions ==================================
        
data = pd.read_parquet(test_file, columns= ['customer_ID'] + feats, engine='pyarrow')
cust_ids = data['customer_ID'].values
all_preds = []

for i, params in enumerate(params_list):
    batch_size = params['batch_size']
    test_loader = load_testdata(scaler_.transform(data[feats]), batch_size=batch_size, n_workers=0, shuffle=False)
    
    for seed in [1, 2001]:
        mlp_preds = inference_mlp(params, f'{i}_mlp_{seed}.pth', test_loader, seed)
        all_preds.append(mlp_preds)
        
test_preds = pd.DataFrame(data={'customer_ID': data['customer_ID'].values, 'prediction':all_preds.mean(axis=0)})
test_preds.to_csv('predictions/mlp.csv', index=False)

####==================4. clean up memory ==================================
del data, all_preds, mlp_preds, test_loader
gc.collect()

## AE MLP

In [ ]:
#https://pytorch-forecasting.readthedocs.io/en/latest/_modules/pytorch_forecasting/models/mlp/submodules.html#FullyConnectedModule
#https://www.kaggle.com/c/jane-street-market-prediction/discussion/224348
#https://www.kaggle.com/code/gogo827jz/jane-street-supervised-autoencoder-mlp/notebook?scriptVersionId=73762661

import torch
from torch import nn
import numpy as np


class AE_MLP(nn.Module):
    def __init__(
        self,
        input_size: int,
        output_size: int,
        hidden_sizes: list,
        dropouts: list,
    ):
        super().__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_sizes = hidden_sizes
        self.dropouts = dropouts
        
        #----normalize input data--------------
        self.bn0 = nn.BatchNorm1d(input_size)
        
        #---encoder layer----------------
        self.encoder = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]), 
                                     nn.BatchNorm1d(hidden_sizes[0]), 
                                     nn.SiLU()
                                    )
        #---decoder layer----------------
        self.decoder = nn.Sequential(nn.Dropout(dropouts[0]), 
                                     nn.Linear(hidden_sizes[0], input_size) 
                                    )
        #----AE output layer-------------
        self.ae_out = nn.Sequential(nn.Linear(input_size, hidden_sizes[1]), 
                                    nn.BatchNorm1d(hidden_sizes[1]), 
                                    nn.SiLU(), 
                                    nn.Dropout(dropouts[1]), 
                                    nn.Linear(hidden_sizes[1], output_size),
                                    nn.Sigmoid(), #for binary classification loss function BCELoss
                                    )
        #---MLP--------------------------
                
        # input layer
        size2 = input_size+hidden_sizes[0]
        module_list = [nn.BatchNorm1d(size2), 
                       nn.Dropout(dropouts[2]), 
                       nn.Linear(size2, hidden_sizes[2]), 
                       nn.BatchNorm1d(hidden_sizes[2]), 
                       nn.SiLU(), 
                       nn.Dropout(dropouts[2])]
    
        # hidden layers
        for i in range(3, len(hidden_sizes)):
            module_list.extend([nn.Linear(hidden_sizes[i-1], hidden_sizes[i]), 
                                nn.BatchNorm1d(hidden_sizes[i]), 
                                nn.SiLU(), 
                                nn.Dropout(dropouts[i])]
                              )
        # output layer
        module_list.extend([nn.Linear(hidden_sizes[-1], output_size), 
                           nn.Sigmoid()])

        self.mlp = nn.Sequential(*module_list)
        
    def forward(self, x):
        x0 = self.bn0(x)
        encoder = self.encoder(x0)
        decoder = self.decoder(encoder)
        out_ae = self.ae_out(decoder)
        
        #x0 shape is n*m - n samples, m features
        #encoder is n*k - n samples, k features
        #x1 is n*(k+m) - n samples, (k+m) features
        #if x0 is n*w*m with w as the width for 3d array, the output will be n*w*(k+m)
        x1 = torch.cat((x0, encoder), dim = -1) #
        out = self.mlp(x1)

        return decoder, out_ae, out


In [ ]:
def train_aemlp(params, seed, train_loader, model_file):
    pl.seed_everything(seed)
    print(params)
        
    learning_rate = params['learning_rate']
    num_epochs = params['num_epochs']
    batch_size = params['batch_size']
    n_layers = params['n_layers']
    h_sizes = [params[f'hidden_size{i}'] for i in range(1,9)][:(n_layers + 3)]
    drop_list = [params[f'dropout{i}'] for i in range(1,9)][:(n_layers + 3)]
    
    model = AE_MLP(input_size=len(feats), output_size=1, 
                   hidden_sizes=h_sizes, 
                   dropouts = drop_list)

    model = model.to(device)

    # optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)  
    optimizer = torch.optim.RMSprop([
            {'params': model.encoder.parameters()},
            {'params': model.decoder.parameters()},
            {'params': model.ae_out.parameters()},
            {'params': model.mlp.parameters()},
        ], lr=learning_rate)

    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e3, 
                                              max_lr=1e-2, epochs=num_epochs, steps_per_epoch=len(train_loader))

    decoder_loss = MSELoss()
    out_ae_loss = BCELoss()
    out_loss = BCELoss()


    #------train models--------------------------
    for epoch in range(num_epochs):
        model.train()
        for batch_idx, (features, targets) in enumerate(train_loader):
            optimizer.zero_grad()
            features = features.to(device)
            targets = targets.to(device)

            ### FORWARD AND BACK PROP
            decoder, out_ae, out = model(features)
            decoder_cost = decoder_loss(decoder, features)
            out_ae_cost = out_ae_loss(out_ae.squeeze(), targets)  
            out_cost = out_loss(out.squeeze(), targets) #squeeze the n_samples*1 2d array to 1d array of n_samples
            total_cost = (decoder_cost + out_ae_cost + out_cost)/3

            total_cost.backward()

            ### UPDATE MODEL PARAMETERS
            optimizer.step()
            scheduler.step()
            

    #-------save model to a file-----
    torch.save(model.state_dict(), model_file)
    
    
def inference_aemlp(params, seed, test_loader, model_file):
    #-----load model file
    pl.seed_everything(seed)
    
    learning_rate = params['learning_rate']
    num_epochs = params['num_epochs']
    batch_size = params['batch_size']
    n_layers = params['n_layers']
    h_sizes = [params[f'hidden_size{i}'] for i in range(1,9)][:(n_layers + 3)]
    drop_list = [params[f'dropout{i}'] for i in range(1,9)][:(n_layers + 3)]
    
    model = AE_MLP(input_size=len(feats), output_size=1, 
                   hidden_sizes=h_sizes, 
                   dropouts = drop_list)
    
    model.load_state_dict(torch.load(model_file, map_location = torch.device('cpu')))

    model = model.to(device)

    
    #-----eval models-------------------------------
    model.eval()

    y_preds = []
    with torch.no_grad():
        for features in test_loader:
            features = features.to(device)
            outputs = model(features)
            y_preds.extend(outputs.squeeze().cpu().numpy())


    y_preds = np.array(y_preds)
    return y_preds

In [ ]:
%%time

aemlp_params =[
    {'batch_size': 4096, 'dropout1': 0.489, 'dropout2': 0.254, 'dropout3': 0.006, 'dropout4': 0.002, 'dropout5': 0.372, 'dropout6': 0.375, 'dropout7': 0.131, 'dropout8': 0.496, 'hidden_size1': 1024, 'hidden_size2': 512, 'hidden_size3': 48, 'hidden_size4': 512, 'hidden_size5': 48, 'hidden_size6': 896, 'hidden_size7': 256, 'hidden_size8': 256, 'learning_rate': 0.006, 'n_layers': 7, 'num_epochs': 20},
    {'batch_size': 1024, 'dropout1': 0.385, 'dropout2': 0.321, 'dropout3': 0.193, 'dropout4': 0.28, 'dropout5': 0.082, 'dropout6': 0.165, 'dropout7': 0.001, 'dropout8': 0.263, 'hidden_size1': 896, 'hidden_size2': 1024, 'hidden_size3': 1024, 'hidden_size4': 256, 'hidden_size5': 1024, 'hidden_size6': 96, 'hidden_size7': 448, 'hidden_size8': 896, 'learning_rate': 0.0002, 'n_layers': 5, 'num_epochs': 35}, 
]

print(len(aemlp_params))

####==================1. load data===================================

df = pd.read_parquet(train_file, columns=['customer_ID', 'target'] + feats, engine='pyarrow')
#df=pd.read_parquet(train_file, engine='pyarrow')

X = df[feats]
y = df[['target']]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

scaler_ = StandardScaler()
scaler_.fit(X)

####==================2. train models ===================================
for i, params in enumerate(aemlp_params):
    batch_size = params['batch_size']
    train_loader = load_traindata(scaler_.transform(X), y['target'].values, 
                         batch_size=batch_size, n_workers=0, shuffle=False)
    for seed in [1, 2001]:
        train_aemlp(params, seed, train_loader, f'{i}_aemlp_{seed}.pth')

del df, X, y, train_loader
gc.collect()
####==================3. predictions ==================================
        
data = pd.read_parquet(test_file, columns= ['customer_ID'] + feats, engine='pyarrow')
cust_ids = data['customer_ID'].values
all_preds = []

for i, params in enumerate(params_list):
    batch_size = params['batch_size']
    test_loader = load_testdata(scaler_.transform(data[feats]), batch_size=batch_size, n_workers=0, shuffle=False)
    
    for seed in [1, 2001]:
        aemlp_preds = inference_aemlp(params, f'{i}_aemlp_{seed}.pth', test_loader, seed)
        all_preds.append(aemlp_preds)
        
test_preds = pd.DataFrame(data={'customer_ID': data['customer_ID'].values, 'prediction':all_preds.mean(axis=0)})
test_preds.to_csv('predictions/aemlp.csv', index=False)

####==================4. clean up memory ==================================
del data, all_preds, mlp_preds, test_loader
gc.collect()

### 

## final submission

In [ ]:
%%time

submission = pd.read_csv('/kaggle/input/amex-default-prediction/sample_submission.csv')
print(submission.shape)
submission = submission[['customer_ID']].merge(test_preds[['customer_ID', 'prediction']], 
                                               on=['customer_ID'], how='left')

submission.isna().sum()

In [ ]:
submission.fillna(value=0, inplace=True)
submission.to_csv('submission.csv', index=False)

In [ ]:
import numpy as np
np.mean([0.7922402961198833,
0.7877414776509613,
0.7901565286865695,
0.7955517225534025])